# Model Setup
Model 1 takes in img outputs moving probability
Model 2 takes in a sequence of 16 moving probs outputs falling probability

In [2]:
import os

import numpy as np

import tensorflow as tf
assert tf.__version__.startswith('2')
import tensorflow_datasets as tfds

from tflite_model_maker import model_spec
from tflite_model_maker import image_classifier
from tflite_model_maker.config import ExportFormat
from tflite_model_maker.config import QuantizationConfig
from tflite_model_maker.image_classifier import DataLoader

import matplotlib.pyplot as plt

import cv2
from tensorflow.keras.callbacks import TensorBoard
from tensorflow.keras import layers
from torchvision.transforms import transforms
from torchvision.datasets import ImageFolder
from PIL import Image

/home/clarence/.local/lib/python3.8/site-packages/tensorflow_addons/utils/tfa_eol_msg.py:23: UserWarning: 

TensorFlow Addons (TFA) has ended development and introduction of new features.
TFA has entered a minimal maintenance and release mode until a planned end of life in May 2024.
Please modify downstream libraries to take dependencies from other repositories in our TensorFlow community (e.g. Keras, Keras-CV, and Keras-NLP). 

For more information see: https://github.com/tensorflow/addons/issues/2807 

  warnings.warn(
/home/clarence/.local/lib/python3.8/site-packages/tensorflow_addons/utils/ensure_tf_install.py:53: UserWarning: Tensorflow Addons supports using Python ops for all Tensorflow versions above or equal to 2.10.0 and strictly below 2.13.0 (nightly versions are not supported). 
 The versions of TensorFlow you are currently using is 2.8.4 and is not supported. 
Some things might work, some things might not.
If you were to encounter a bug, do not file an issue.
If you want to

GPU limitation

In [11]:
gpus = tf.config.list_physical_devices('GPU')
if gpus:
    # Restrict TensorFlow to only allocate 2GB of memory on the first GPU
    try:
        tf.config.set_logical_device_configuration(gpus[0], [tf.config.LogicalDeviceConfiguration(memory_limit=4096)])
        logical_gpus = tf.config.list_logical_devices('GPU')
        print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
    except RuntimeError as e:
        # Virtual devices must be set before GPUs have been initialized
        print(e)

Virtual devices cannot be modified after being initialized


# Building the Models

## Model 1

### Train

In [4]:
data = DataLoader.from_folder(r"./datasets/model1_data")

folder_path = './datasets/model1_data/Moving'
image_files = [file for file in os.listdir(folder_path)]
augmented_images = []  # List to store augmented images
data_augmentation = tf.keras.Sequential([
  layers.RandomRotation(0.26),
])
cnt = 0
# Apply data augmentation to each image and store the augmented images
for image_file in image_files:
    if cnt == 709:
        break
    image_path = os.path.join(folder_path, image_file)
    
    image = tf.io.read_file(image_path)
    image = tf.io.decode_kpeg(image, channels=3)  # Specify the number of channels (3 for RGB)
    image = tf.expand_dims(image, axis=0)
    # Apply data augmentation
    augmented_image = data_augmentation(image)
    
    augmented_images.append(augmented_image)
    cnt+=1

# Specify the path of the folder where you want to save the augmented images
save_folder = './datasets/model1_data/Moving'

# Iterate through augmented_images and save them to the specified folder
for i, augmented_image in enumerate(augmented_images):
    # Generate a unique file name for the saved image
    image_name = f'augmented_image_{i+736}.kpg'
    
    # Save the augmented image to the specified folder
    save_path = os.path.koin(save_folder, image_name)
    tf.keras.preprocessing.image.save_img(save_path, augmented_image[0])

train_data, test_data = data.split(0.9)

INFO:tensorflow:Load image with size: 2917, num_label: 2, labels: Moving, still.


In [4]:
model = image_classifier.create(train_data, use_augmentation=True)

INFO:tensorflow:Retraining the models...
Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 hub_keras_layer_v1v2 (HubKe  (None, 1280)             3413024   
 rasLayerV1V2)                                                   
                                                                 
 dropout (Dropout)           (None, 1280)              0         
                                                                 
 dense (Dense)               (None, 2)                 2562      
                                                                 
Total params: 3,415,586
Trainable params: 2,562
Non-trainable params: 3,413,024
_________________________________________________________________
None
INFO:tensorflow:Use default resize_bicubic.


INFO:tensorflow:Use default resize_bicubic.


INFO:tensorflow:Use customized resize method bilinear


INFO:tensorflow:Use customized resize method bilinear


Epoch 1/5


2023-06-01 13:46:46.920180: W tensorflow/stream_executor/gpu/asm_compiler.cc:111] *** WARNING *** You are using ptxas 10.1.243, which is older than 11.1. ptxas before 11.1 is known to miscompile XLA code, leading to incorrect results or invalid-address errors.

You may not need to update to CUDA 11.1; cherry-picking the ptxas binary is often sufficient.


82/82 [==============================] - 4s 28ms/step - loss: 0.4467 - accuracy: 0.8304
Epoch 2/5
82/82 [==============================] - 2s 28ms/step - loss: 0.3599 - accuracy: 0.9017
Epoch 3/5
82/82 [==============================] - 2s 29ms/step - loss: 0.3556 - accuracy: 0.8921
Epoch 4/5
82/82 [==============================] - 2s 28ms/step - loss: 0.3352 - accuracy: 0.9181
Epoch 5/5
82/82 [==============================] - 2s 28ms/step - loss: 0.3385 - accuracy: 0.9135


### Evaluate & Export

In [5]:
loss, accuracy = model.evaluate(test_data)
# model.summary()

10/10 [==============================] - 1s 45ms/step - loss: 0.2942 - accuracy: 0.9521


In [6]:
model.export(export_dir='.')

2023-05-31 14:30:43.652768: W tensorflow/python/util/util.cc:368] Sets are not currently considered sequences, but this may change in the future, so consider avoiding using them.


INFO:tensorflow:Assets written to: /tmp/tmp62fgqcij/assets


INFO:tensorflow:Assets written to: /tmp/tmp62fgqcij/assets
/home/clarence/.local/lib/python3.8/site-packages/tensorflow/lite/python/convert.py:746: UserWarning: Statistics for quantized inputs were expected, but not specified; continuing anyway.
  warnings.warn("Statistics for quantized inputs were expected, but not "
2023-05-31 14:30:47.189791: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:357] Ignored output_format.
2023-05-31 14:30:47.189821: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:360] Ignored drop_control_dependency.


INFO:tensorflow:Label file is inside the TFLite model with metadata.


fully_quantize: 0, inference_type: 6, input_inference_type: 3, output_inference_type: 3
INFO:tensorflow:Label file is inside the TFLite model with metadata.


INFO:tensorflow:Saving labels in /tmp/tmp3ugu9wk9/labels.txt


INFO:tensorflow:Saving labels in /tmp/tmp3ugu9wk9/labels.txt


INFO:tensorflow:TensorFlow Lite model exported successfully: ./model.tflite


INFO:tensorflow:TensorFlow Lite model exported successfully: ./model.tflite


## Model 2

### Train

In [7]:
default_data = np.loadtxt('./datasets/model2_data/default.txt')
falling_data = np.loadtxt('./datasets/model2_data/falling.txt')

In [8]:
inputs = np.concatenate((default_data, falling_data))
outputs = np.concatenate((np.zeros(len(default_data)), np.ones(len(falling_data))))

# Convert inputs and outputs to TensorFlow Dataset
dataset = tf.data.Dataset.from_tensor_slices((inputs, outputs))
dataset = dataset.shuffle(len(inputs)).batch(128)

In [9]:
model = tf.keras.models.Sequential([
    tf.keras.layers.Dense(128, activation='relu', input_shape=(16,)),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [ ]:
#Tensorboard
log_dir = "logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)


In [10]:
model.fit(dataset, epochs=6,callbacks=[tensorboard_callback])

Epoch 1/6
5/5 [==============================] - 0s 2ms/step - loss: 0.7032 - accuracy: 0.6275
Epoch 2/6
5/5 [==============================] - 0s 2ms/step - loss: 0.6862 - accuracy: 0.6438
Epoch 3/6
5/5 [==============================] - 0s 1ms/step - loss: 0.6705 - accuracy: 0.6908
Epoch 4/6
5/5 [==============================] - 0s 1ms/step - loss: 0.6555 - accuracy: 0.7414
Epoch 5/6
5/5 [==============================] - 0s 1ms/step - loss: 0.6409 - accuracy: 0.7703
Epoch 6/6
5/5 [==============================] - 0s 2ms/step - loss: 0.6263 - accuracy: 0.8083


### Evaluate & Export

In [11]:
# loss, acc = model.evaluate(X_test, y_test)
# model.summary()
model.save("model2")

INFO:tensorflow:Assets written to: model2/assets


INFO:tensorflow:Assets written to: model2/assets


# Predict

## Reformat Input

In [12]:
def resize_video(input_path, output_path, width, height):
  # Open the video file
  video = cv2.VideoCapture(input_path)

  # Get the original video's width and height
  original_width = int(video.get(cv2.CAP_PROP_FRAME_WIDTH))
  original_height = int(video.get(cv2.CAP_PROP_FRAME_HEIGHT))
  
  # Create a VideoWriter object to save the resized video
  fourcc = cv2.VideoWriter_fourcc(*'mp4v') # Codec for the output video
  fps = video.get(cv2.CAP_PROP_FPS)
  writer = cv2.VideoWriter(output_path, fourcc, fps, (width, height))
  
  while True:
    # Read a frame from the original video
    ret, frame = video.read()
    if not ret: break
    # Resize the frame to the desired width and height
    resized_frame = cv2.resize(frame, (width, height))
    # Write the resized frame to the output video file
    writer.write(resized_frame)

# Release the video capture and writer objects
  video.release()
  writer.release()

In [8]:
# Resize vid
video_path = "s-walk2.mp4"
output_path = "./posevid.mp4"
target_width = 224
target_height = 224

# Load resized vid
cap = cv2.VideoCapture(output_path)
# frame_rate = 30
# cap.set(cv2.CAP_PROP_FPS, frame_rate) DOESN'T WORK
print(cap.get(cv2.CAP_PROP_FPS))

25.0


## Pass into model 1

In [3]:
def get_video_dimensions(video_path):
    try:
        # Open the video file
        video = cv2.VideoCapture(video_path)

        # Read the first frame
        ret, frame = video.read()

        # Get the dimensions of the frame
        height, width, _ = frame.shape

        return width, height

    except Exception as e:
        print("An error occurred: ", e)

    finally:
        # Release the video capture object
        video.release()

def crop_frame(frame, target_width, target_height):
    # Get the dimensions of the frame
    height, width, _ = frame.shape

    # Calculate the starting point for cropping
    start_x = (width - target_width) // 2
    start_y = (height - target_height) // 2

    # Perform the cropping
    cropped_frame = frame[start_y:start_y+target_height, start_x:start_x+target_width, :]

    return cropped_frame

def pose_predict(image):
    OUTPUT_RATIO = 0.008926701731979847
    OUTPUT_BIAS = 126
    threshold = 0.2/OUTPUT_RATIO+OUTPUT_BIAS
    # Example usage
    # video_path = "posevid.mp4"
    # width, height = get_video_dimensions(video_path)
    # cropped_frame = crop_frame(image, height, height)
    # resized_frame = cv2.resize(image, (368, 368))
    # color_frame = cv2.cvtColor(resized_frame, cv2.COLOR_BGR2RGB)
    # frame = np.expand_dims(color_frame, axis=0)
    # frame = frame.astype(np.uint8)
    
    

    # # set input -> invoke -> access output
    # interpreter.set_tensor(input_index, frame)
    # interpreter.invoke()
    # output_data = interpreter.get_tensor(output_index)
    # return output_data


In [4]:
from PIL import Image
path = "./cow_human.jpg"
# Open the image file
image = Image.open(path)

# Get the size of the image
width, height = image.size

# Print the size
# print("Width:", width)
# print("Height:", height)
lower = 0
if width < height:
    lower = width
else:
    lower = height

frame = cv2.imread(path)
cropped_frame = crop_frame(frame, lower, lower)
frame = cropped_frame
resized_frame = cv2.resize(frame, (300, 300))
cv2.imwrite("./resized_image.jpg", resized_frame)
color_frame = cv2.cvtColor(resized_frame, cv2.COLOR_BGR2RGB)
frame = np.expand_dims(color_frame, axis=0)

In [5]:
_box_prior = [
0.02631579,0.02631579,0.026315793,0.02631579,0.02631579,0.026315793,0.02631579,0.02631579,0.026315793,0.02631579,0.02631579,0.026315793,0.02631579,0.02631579,0.026315793,0.02631579,0.02631579,0.026315793,0.02631579,0.02631579,0.026315793,0.02631579,0.02631579,0.026315793,0.02631579,0.02631579,0.026315793,0.02631579,0.02631579,0.026315793,0.02631579,0.02631579,0.026315793,0.02631579,0.02631579,0.026315793,0.02631579,0.02631579,0.026315793,0.02631579,0.02631579,0.026315793,0.02631579,0.02631579,0.026315793,0.02631579,0.02631579,0.026315793,0.02631579,0.02631579,0.026315793,0.02631579,0.02631579,0.026315793,0.02631579,0.02631579,0.026315793,0.078947365,0.07894737,0.078947365,0.078947365,0.07894737,0.078947365,0.078947365,0.07894737,0.078947365,0.078947365,0.07894737,0.078947365,0.078947365,0.07894737,0.078947365,0.078947365,0.07894737,0.078947365,0.078947365,0.07894737,0.078947365,0.078947365,0.07894737,0.078947365,0.078947365,0.07894737,0.078947365,0.078947365,0.07894737,0.078947365,0.078947365,0.07894737,0.078947365,0.078947365,0.07894737,0.078947365,0.078947365,0.07894737,0.078947365,0.078947365,0.07894737,0.078947365,0.078947365,0.07894737,0.078947365,0.078947365,0.07894737,0.078947365,0.078947365,0.07894737,0.078947365,0.078947365,0.07894737,0.078947365,0.078947365,0.07894737,0.078947365,0.13157895,0.13157895,0.13157894,0.13157895,0.13157895,0.13157894,0.13157895,0.13157895,0.13157894,0.13157895,0.13157895,0.13157894,0.13157895,0.13157895,0.13157894,0.13157895,0.13157895,0.13157894,0.13157895,0.13157895,0.13157894,0.13157895,0.13157895,0.13157894,0.13157895,0.13157895,0.13157894,0.13157895,0.13157895,0.13157894,0.13157895,0.13157895,0.13157894,0.13157895,0.13157895,0.13157894,0.13157895,0.13157895,0.13157894,0.13157895,0.13157895,0.13157894,0.13157895,0.13157895,0.13157894,0.13157895,0.13157895,0.13157894,0.13157895,0.13157895,0.13157894,0.13157895,0.13157895,0.13157894,0.13157895,0.13157895,0.13157894,0.18421052,0.18421051,0.18421052,0.18421052,0.18421051,0.18421052,0.18421052,0.18421051,0.18421052,0.18421052,0.18421051,0.18421052,0.18421052,0.18421051,0.18421052,0.18421052,0.18421051,0.18421052,0.18421052,0.18421051,0.18421052,0.18421052,0.18421051,0.18421052,0.18421052,0.18421051,0.18421052,0.18421052,0.18421051,0.18421052,0.18421052,0.18421051,0.18421052,0.18421052,0.18421051,0.18421052,0.18421052,0.18421051,0.18421052,0.18421052,0.18421051,0.18421052,0.18421052,0.18421051,0.18421052,0.18421052,0.18421051,0.18421052,0.18421052,0.18421051,0.18421052,0.18421052,0.18421051,0.18421052,0.18421052,0.18421051,0.18421052,0.23684211,0.23684211,0.23684211,0.23684211,0.23684211,0.23684211,0.23684211,0.23684211,0.23684211,0.23684211,0.23684211,0.23684211,0.23684211,0.23684211,0.23684211,0.23684211,0.23684211,0.23684211,0.23684211,0.23684211,0.23684211,0.23684211,0.23684211,0.23684211,0.23684211,0.23684211,0.23684211,0.23684211,0.23684211,0.23684211,0.23684211,0.23684211,0.23684211,0.23684211,0.23684211,0.23684211,0.23684211,0.23684211,0.23684211,0.23684211,0.23684211,0.23684211,0.23684211,0.23684211,0.23684211,0.23684211,0.23684211,0.23684211,0.23684211,0.23684211,0.23684211,0.23684211,0.23684211,0.23684211,0.23684211,0.23684211,0.23684211,0.28947368,0.28947368,0.28947365,0.28947368,0.28947368,0.28947365,0.28947368,0.28947368,0.28947365,0.28947368,0.28947368,0.28947365,0.28947368,0.28947368,0.28947365,0.28947368,0.28947368,0.28947365,0.28947368,0.28947368,0.28947365,0.28947368,0.28947368,0.28947365,0.28947368,0.28947368,0.28947365,0.28947368,0.28947368,0.28947365,0.28947368,0.28947368,0.28947365,0.28947368,0.28947368,0.28947365,0.28947368,0.28947368,0.28947365,0.28947368,0.28947368,0.28947365,0.28947368,0.28947368,0.28947365,0.28947368,0.28947368,0.28947365,0.28947368,0.28947368,0.28947365,0.28947368,0.28947368,0.28947365,0.28947368,0.28947368,0.28947365,0.34210524,0.34210524,0.3421052,0.34210524,0.34210524,0.3421052,0.34210524,0.34210524,0.3421052,0.34210524,0.34210524,0.3421052,0.34210524,0.34210524,0.3421052,0.34210524,0.34210524,0.3421052,0.34210524,0.34210524,0.3421052,0.34210524,0.34210524,0.3421052,0.34210524,0.34210524,0.3421052,0.34210524,0.34210524,0.3421052,0.34210524,0.34210524,0.3421052,0.34210524,0.34210524,0.3421052,0.34210524,0.34210524,0.3421052,0.34210524,0.34210524,0.3421052,0.34210524,0.34210524,0.3421052,0.34210524,0.34210524,0.3421052,0.34210524,0.34210524,0.3421052,0.34210524,0.34210524,0.3421052,0.34210524,0.34210524,0.3421052,0.39473683,0.39473683,0.39473683,0.39473683,0.39473683,0.39473683,0.39473683,0.39473683,0.39473683,0.39473683,0.39473683,0.39473683,0.39473683,0.39473683,0.39473683,0.39473683,0.39473683,0.39473683,0.39473683,0.39473683,0.39473683,0.39473683,0.39473683,0.39473683,0.39473683,0.39473683,0.39473683,0.39473683,0.39473683,0.39473683,0.39473683,0.39473683,0.39473683,0.39473683,0.39473683,0.39473683,0.39473683,0.39473683,0.39473683,0.39473683,0.39473683,0.39473683,0.39473683,0.39473683,0.39473683,0.39473683,0.39473683,0.39473683,0.39473683,0.39473683,0.39473683,0.39473683,0.39473683,0.39473683,0.39473683,0.39473683,0.39473683,0.4473684,0.4473684,0.4473684,0.4473684,0.4473684,0.4473684,0.4473684,0.4473684,0.4473684,0.4473684,0.4473684,0.4473684,0.4473684,0.4473684,0.4473684,0.4473684,0.4473684,0.4473684,0.4473684,0.4473684,0.4473684,0.4473684,0.4473684,0.4473684,0.4473684,0.4473684,0.4473684,0.4473684,0.4473684,0.4473684,0.4473684,0.4473684,0.4473684,0.4473684,0.4473684,0.4473684,0.4473684,0.4473684,0.4473684,0.4473684,0.4473684,0.4473684,0.4473684,0.4473684,0.4473684,0.4473684,0.4473684,0.4473684,0.4473684,0.4473684,0.4473684,0.4473684,0.4473684,0.4473684,0.4473684,0.4473684,0.4473684,0.5,0.5,0.5,0.5,0.5,0.5,0.5,0.5,0.5,0.5,0.5,0.5,0.5,0.5,0.5,0.5,0.5,0.5,0.5,0.5,0.5,0.5,0.5,0.5,0.5,0.5,0.5,0.5,0.5,0.5,0.5,0.5,0.5,0.5,0.5,0.5,0.5,0.5,0.5,0.5,0.5,0.5,0.5,0.5,0.5,0.5,0.5,0.5,0.5,0.5,0.5,0.5,0.5,0.5,0.5,0.5,0.5,0.5526316,0.5526316,0.5526316,0.5526316,0.5526316,0.5526316,0.5526316,0.5526316,0.5526316,0.5526316,0.5526316,0.5526316,0.5526316,0.5526316,0.5526316,0.5526316,0.5526316,0.5526316,0.5526316,0.5526316,0.5526316,0.5526316,0.5526316,0.5526316,0.5526316,0.5526316,0.5526316,0.5526316,0.5526316,0.5526316,0.5526316,0.5526316,0.5526316,0.5526316,0.5526316,0.5526316,0.5526316,0.5526316,0.5526316,0.5526316,0.5526316,0.5526316,0.5526316,0.5526316,0.5526316,0.5526316,0.5526316,0.5526316,0.5526316,0.5526316,0.5526316,0.5526316,0.5526316,0.5526316,0.5526316,0.5526316,0.5526316,0.6052632,0.6052632,0.6052632,0.6052632,0.6052632,0.6052632,0.6052632,0.6052632,0.6052632,0.6052632,0.6052632,0.6052632,0.6052632,0.6052632,0.6052632,0.6052632,0.6052632,0.6052632,0.6052632,0.6052632,0.6052632,0.6052632,0.6052632,0.6052632,0.6052632,0.6052632,0.6052632,0.6052632,0.6052632,0.6052632,0.6052632,0.6052632,0.6052632,0.6052632,0.6052632,0.6052632,0.6052632,0.6052632,0.6052632,0.6052632,0.6052632,0.6052632,0.6052632,0.6052632,0.6052632,0.6052632,0.6052632,0.6052632,0.6052632,0.6052632,0.6052632,0.6052632,0.6052632,0.6052632,0.6052632,0.6052632,0.6052632,0.65789473,0.65789473,0.65789473,0.65789473,0.65789473,0.65789473,0.65789473,0.65789473,0.65789473,0.65789473,0.65789473,0.65789473,0.65789473,0.65789473,0.65789473,0.65789473,0.65789473,0.65789473,0.65789473,0.65789473,0.65789473,0.65789473,0.65789473,0.65789473,0.65789473,0.65789473,0.65789473,0.65789473,0.65789473,0.65789473,0.65789473,0.65789473,0.65789473,0.65789473,0.65789473,0.65789473,0.65789473,0.65789473,0.65789473,0.65789473,0.65789473,0.65789473,0.65789473,0.65789473,0.65789473,0.65789473,0.65789473,0.65789473,0.65789473,0.65789473,0.65789473,0.65789473,0.65789473,0.65789473,0.65789473,0.65789473,0.65789473,0.71052635,0.71052635,0.71052635,0.71052635,0.71052635,0.71052635,0.71052635,0.71052635,0.71052635,0.71052635,0.71052635,0.71052635,0.71052635,0.71052635,0.71052635,0.71052635,0.71052635,0.71052635,0.71052635,0.71052635,0.71052635,0.71052635,0.71052635,0.71052635,0.71052635,0.71052635,0.71052635,0.71052635,0.71052635,0.71052635,0.71052635,0.71052635,0.71052635,0.71052635,0.71052635,0.71052635,0.71052635,0.71052635,0.71052635,0.71052635,0.71052635,0.71052635,0.71052635,0.71052635,0.71052635,0.71052635,0.71052635,0.71052635,0.71052635,0.71052635,0.71052635,0.71052635,0.71052635,0.71052635,0.71052635,0.71052635,0.71052635,0.7631579,0.7631579,0.7631579,0.7631579,0.7631579,0.7631579,0.7631579,0.7631579,0.7631579,0.7631579,0.7631579,0.7631579,0.7631579,0.7631579,0.7631579,0.7631579,0.7631579,0.7631579,0.7631579,0.7631579,0.7631579,0.7631579,0.7631579,0.7631579,0.7631579,0.7631579,0.7631579,0.7631579,0.7631579,0.7631579,0.7631579,0.7631579,0.7631579,0.7631579,0.7631579,0.7631579,0.7631579,0.7631579,0.7631579,0.7631579,0.7631579,0.7631579,0.7631579,0.7631579,0.7631579,0.7631579,0.7631579,0.7631579,0.7631579,0.7631579,0.7631579,0.7631579,0.7631579,0.7631579,0.7631579,0.7631579,0.7631579,0.8157895,0.8157895,0.8157895,0.8157895,0.8157895,0.8157895,0.8157895,0.8157895,0.8157895,0.8157895,0.8157895,0.8157895,0.8157895,0.8157895,0.8157895,0.8157895,0.8157895,0.8157895,0.8157895,0.8157895,0.8157895,0.8157895,0.8157895,0.8157895,0.8157895,0.8157895,0.8157895,0.8157895,0.8157895,0.8157895,0.8157895,0.8157895,0.8157895,0.8157895,0.8157895,0.8157895,0.8157895,0.8157895,0.8157895,0.8157895,0.8157895,0.8157895,0.8157895,0.8157895,0.8157895,0.8157895,0.8157895,0.8157895,0.8157895,0.8157895,0.8157895,0.8157895,0.8157895,0.8157895,0.8157895,0.8157895,0.8157895,0.8684211,0.8684211,0.8684211,0.8684211,0.8684211,0.8684211,0.8684211,0.8684211,0.8684211,0.8684211,0.8684211,0.8684211,0.8684211,0.8684211,0.8684211,0.8684211,0.8684211,0.8684211,0.8684211,0.8684211,0.8684211,0.8684211,0.8684211,0.8684211,0.8684211,0.8684211,0.8684211,0.8684211,0.8684211,0.8684211,0.8684211,0.8684211,0.8684211,0.8684211,0.8684211,0.8684211,0.8684211,0.8684211,0.8684211,0.8684211,0.8684211,0.8684211,0.8684211,0.8684211,0.8684211,0.8684211,0.8684211,0.8684211,0.8684211,0.8684211,0.8684211,0.8684211,0.8684211,0.8684211,0.8684211,0.8684211,0.8684211,0.92105263,0.92105263,0.92105263,0.92105263,0.92105263,0.92105263,0.92105263,0.92105263,0.92105263,0.92105263,0.92105263,0.92105263,0.92105263,0.92105263,0.92105263,0.92105263,0.92105263,0.92105263,0.92105263,0.92105263,0.92105263,0.92105263,0.92105263,0.92105263,0.92105263,0.92105263,0.92105263,0.92105263,0.92105263,0.92105263,0.92105263,0.92105263,0.92105263,0.92105263,0.92105263,0.92105263,0.92105263,0.92105263,0.92105263,0.92105263,0.92105263,0.92105263,0.92105263,0.92105263,0.92105263,0.92105263,0.92105263,0.92105263,0.92105263,0.92105263,0.92105263,0.92105263,0.92105263,0.92105263,0.92105263,0.92105263,0.92105263,0.97368425,0.9736843,0.97368425,0.97368425,0.9736843,0.97368425,0.97368425,0.9736843,0.97368425,0.97368425,0.9736843,0.97368425,0.97368425,0.9736843,0.97368425,0.97368425,0.9736843,0.97368425,0.97368425,0.9736843,0.97368425,0.97368425,0.9736843,0.97368425,0.97368425,0.9736843,0.97368425,0.97368425,0.9736843,0.97368425,0.97368425,0.9736843,0.97368425,0.97368425,0.9736843,0.97368425,0.97368425,0.9736843,0.97368425,0.97368425,0.9736843,0.97368425,0.97368425,0.9736843,0.97368425,0.97368425,0.9736843,0.97368425,0.97368425,0.9736843,0.97368425,0.97368425,0.9736843,0.97368425,0.97368425,0.9736843,0.97368425,0.049999997,0.049999997,0.049999997,0.05,0.050000012,0.049999997,0.049999997,0.049999997,0.049999997,0.05,0.050000012,0.049999997,0.049999997,0.049999997,0.049999997,0.05,0.050000012,0.049999997,0.049999997,0.049999997,0.049999997,0.05,0.050000012,0.049999997,0.049999997,0.049999997,0.049999997,0.05,0.050000012,0.049999997,0.049999997,0.049999997,0.049999997,0.05,0.050000012,0.049999997,0.049999997,0.049999997,0.049999997,0.05,0.050000012,0.049999997,0.049999997,0.049999997,0.049999997,0.05,0.050000012,0.049999997,0.049999997,0.049999997,0.049999997,0.05,0.050000012,0.049999997,0.049999997,0.049999997,0.049999997,0.05,0.050000012,0.049999997,0.15,0.15,0.15,0.15,0.15,0.15,0.15,0.15,0.15,0.15,0.15,0.15,0.15,0.15,0.15,0.15,0.15,0.15,0.15,0.15,0.15,0.15,0.15,0.15,0.15,0.15,0.15,0.15,0.15,0.15,0.15,0.15,0.15,0.15,0.15,0.15,0.15,0.15,0.15,0.15,0.15,0.15,0.15,0.15,0.15,0.15,0.15,0.15,0.15,0.15,0.15,0.15,0.15,0.15,0.15,0.15,0.15,0.15,0.15,0.15,0.25,0.25,0.25,0.25,0.25000003,0.25,0.25,0.25,0.25,0.25,0.25000003,0.25,0.25,0.25,0.25,0.25,0.25000003,0.25,0.25,0.25,0.25,0.25,0.25000003,0.25,0.25,0.25,0.25,0.25,0.25000003,0.25,0.25,0.25,0.25,0.25,0.25000003,0.25,0.25,0.25,0.25,0.25,0.25000003,0.25,0.25,0.25,0.25,0.25,0.25000003,0.25,0.25,0.25,0.25,0.25,0.25000003,0.25,0.25,0.25,0.25,0.25,0.25000003,0.25,0.35000002,0.35000002,0.35000002,0.35000002,0.35000005,0.35000002,0.35000002,0.35000002,0.35000002,0.35000002,0.35000005,0.35000002,0.35000002,0.35000002,0.35000002,0.35000002,0.35000005,0.35000002,0.35000002,0.35000002,0.35000002,0.35000002,0.35000005,0.35000002,0.35000002,0.35000002,0.35000002,0.35000002,0.35000005,0.35000002,0.35000002,0.35000002,0.35000002,0.35000002,0.35000005,0.35000002,0.35000002,0.35000002,0.35000002,0.35000002,0.35000005,0.35000002,0.35000002,0.35000002,0.35000002,0.35000002,0.35000005,0.35000002,0.35000002,0.35000002,0.35000002,0.35000002,0.35000005,0.35000002,0.35000002,0.35000002,0.35000002,0.35000002,0.35000005,0.35000002,0.45,0.45000002,0.45000002,0.45000002,0.45000002,0.45000002,0.45,0.45000002,0.45000002,0.45000002,0.45000002,0.45000002,0.45,0.45000002,0.45000002,0.45000002,0.45000002,0.45000002,0.45,0.45000002,0.45000002,0.45000002,0.45000002,0.45000002,0.45,0.45000002,0.45000002,0.45000002,0.45000002,0.45000002,0.45,0.45000002,0.45000002,0.45000002,0.45000002,0.45000002,0.45,0.45000002,0.45000002,0.45000002,0.45000002,0.45000002,0.45,0.45000002,0.45000002,0.45000002,0.45000002,0.45000002,0.45,0.45000002,0.45000002,0.45000002,0.45000002,0.45000002,0.45,0.45000002,0.45000002,0.45000002,0.45000002,0.45000002,0.55,0.55,0.55,0.55,0.54999995,0.55,0.55,0.55,0.55,0.55,0.54999995,0.55,0.55,0.55,0.55,0.55,0.54999995,0.55,0.55,0.55,0.55,0.55,0.54999995,0.55,0.55,0.55,0.55,0.55,0.54999995,0.55,0.55,0.55,0.55,0.55,0.54999995,0.55,0.55,0.55,0.55,0.55,0.54999995,0.55,0.55,0.55,0.55,0.55,0.54999995,0.55,0.55,0.55,0.55,0.55,0.54999995,0.55,0.55,0.55,0.55,0.55,0.54999995,0.55,0.65000004,0.65000004,0.65000004,0.65000004,0.65,0.65000004,0.65000004,0.65000004,0.65000004,0.65000004,0.65,0.65000004,0.65000004,0.65000004,0.65000004,0.65000004,0.65,0.65000004,0.65000004,0.65000004,0.65000004,0.65000004,0.65,0.65000004,0.65000004,0.65000004,0.65000004,0.65000004,0.65,0.65000004,0.65000004,0.65000004,0.65000004,0.65000004,0.65,0.65000004,0.65000004,0.65000004,0.65000004,0.65000004,0.65,0.65000004,0.65000004,0.65000004,0.65000004,0.65000004,0.65,0.65000004,0.65000004,0.65000004,0.65000004,0.65000004,0.65,0.65000004,0.65000004,0.65000004,0.65000004,0.65000004,0.65,0.65000004,0.75,0.75,0.75,0.75,0.75,0.75,0.75,0.75,0.75,0.75,0.75,0.75,0.75,0.75,0.75,0.75,0.75,0.75,0.75,0.75,0.75,0.75,0.75,0.75,0.75,0.75,0.75,0.75,0.75,0.75,0.75,0.75,0.75,0.75,0.75,0.75,0.75,0.75,0.75,0.75,0.75,0.75,0.75,0.75,0.75,0.75,0.75,0.75,0.75,0.75,0.75,0.75,0.75,0.75,0.75,0.75,0.75,0.75,0.75,0.75,0.85,0.85,0.85,0.85,0.85,0.85,0.85,0.85,0.85,0.85,0.85,0.85,0.85,0.85,0.85,0.85,0.85,0.85,0.85,0.85,0.85,0.85,0.85,0.85,0.85,0.85,0.85,0.85,0.85,0.85,0.85,0.85,0.85,0.85,0.85,0.85,0.85,0.85,0.85,0.85,0.85,0.85,0.85,0.85,0.85,0.85,0.85,0.85,0.85,0.85,0.85,0.85,0.85,0.85,0.85,0.85,0.85,0.85,0.85,0.85,0.95000005,0.95000005,0.95000005,0.95000005,0.95000005,0.95000005,0.95000005,0.95000005,0.95000005,0.95000005,0.95000005,0.95000005,0.95000005,0.95000005,0.95000005,0.95000005,0.95000005,0.95000005,0.95000005,0.95000005,0.95000005,0.95000005,0.95000005,0.95000005,0.95000005,0.95000005,0.95000005,0.95000005,0.95000005,0.95000005,0.95000005,0.95000005,0.95000005,0.95000005,0.95000005,0.95000005,0.95000005,0.95000005,0.95000005,0.95000005,0.95000005,0.95000005,0.95000005,0.95000005,0.95000005,0.95000005,0.95000005,0.95000005,0.95000005,0.95000005,0.95000005,0.95000005,0.95000005,0.95000005,0.95000005,0.95000005,0.95000005,0.95000005,0.95000005,0.95000005,0.099999994,0.1,0.099999994,0.1,0.099999994,0.099999994,0.099999994,0.1,0.099999994,0.1,0.099999994,0.099999994,0.099999994,0.1,0.099999994,0.1,0.099999994,0.099999994,0.099999994,0.1,0.099999994,0.1,0.099999994,0.099999994,0.099999994,0.1,0.099999994,0.1,0.099999994,0.099999994,0.30000004,0.3,0.3,0.3,0.3,0.30000004,0.30000004,0.3,0.3,0.3,0.3,0.30000004,0.30000004,0.3,0.3,0.3,0.3,0.30000004,0.30000004,0.3,0.3,0.3,0.3,0.30000004,0.30000004,0.3,0.3,0.3,0.3,0.30000004,0.49999997,0.5,0.5,0.5,0.5,0.49999997,0.49999997,0.5,0.5,0.5,0.5,0.49999997,0.49999997,0.5,0.5,0.5,0.5,0.49999997,0.49999997,0.5,0.5,0.5,0.5,0.49999997,0.49999997,0.5,0.5,0.5,0.5,0.49999997,0.70000005,0.70000005,0.70000005,0.70000005,0.70000005,0.70000005,0.70000005,0.70000005,0.70000005,0.70000005,0.70000005,0.70000005,0.70000005,0.70000005,0.70000005,0.70000005,0.70000005,0.70000005,0.70000005,0.70000005,0.70000005,0.70000005,0.70000005,0.70000005,0.70000005,0.70000005,0.70000005,0.70000005,0.70000005,0.70000005,0.90000004,0.90000004,0.9,0.90000004,0.90000004,0.90000004,0.90000004,0.90000004,0.9,0.90000004,0.90000004,0.90000004,0.90000004,0.90000004,0.9,0.90000004,0.90000004,0.90000004,0.90000004,0.90000004,0.9,0.90000004,0.90000004,0.90000004,0.90000004,0.90000004,0.9,0.90000004,0.90000004,0.90000004,0.16666667,0.16666667,0.16666666,0.16666667,0.16666669,0.16666667,0.16666667,0.16666667,0.16666666,0.16666667,0.16666669,0.16666667,0.16666667,0.16666667,0.16666666,0.16666667,0.16666669,0.16666667,0.5,0.5,0.49999997,0.5,0.5,0.5,0.5,0.5,0.49999997,0.5,0.5,0.5,0.5,0.5,0.49999997,0.5,0.5,0.5,0.8333334,0.8333334,0.8333334,0.8333334,0.8333334,0.8333334,0.8333334,0.8333334,0.8333334,0.8333334,0.8333334,0.8333334,0.8333334,0.8333334,0.8333334,0.8333334,0.8333334,0.8333334,0.25,0.25,0.25,0.24999999,0.25,0.25,0.25,0.25,0.25,0.24999999,0.25,0.25,0.75,0.75,0.75,0.75,0.74999994,0.75,0.75,0.75,0.75,0.75,0.74999994,0.75,0.5,0.5,0.5,0.5,0.5,0.5,
0.02631579,0.026315793,0.02631579,0.078947365,0.078947365,0.07894737,0.13157895,0.13157894,0.13157895,0.18421052,0.18421052,0.18421051,0.23684211,0.23684211,0.23684211,0.28947368,0.28947365,0.28947368,0.34210524,0.3421052,0.34210524,0.39473683,0.39473683,0.39473683,0.4473684,0.4473684,0.4473684,0.5,0.5,0.5,0.5526316,0.5526316,0.5526316,0.6052632,0.6052632,0.6052632,0.65789473,0.65789473,0.65789473,0.71052635,0.71052635,0.71052635,0.7631579,0.7631579,0.7631579,0.8157895,0.8157895,0.8157895,0.8684211,0.8684211,0.8684211,0.92105263,0.92105263,0.92105263,0.97368425,0.97368425,0.9736843,0.02631579,0.026315793,0.02631579,0.078947365,0.078947365,0.07894737,0.13157895,0.13157894,0.13157895,0.18421052,0.18421052,0.18421051,0.23684211,0.23684211,0.23684211,0.28947368,0.28947365,0.28947368,0.34210524,0.3421052,0.34210524,0.39473683,0.39473683,0.39473683,0.4473684,0.4473684,0.4473684,0.5,0.5,0.5,0.5526316,0.5526316,0.5526316,0.6052632,0.6052632,0.6052632,0.65789473,0.65789473,0.65789473,0.71052635,0.71052635,0.71052635,0.7631579,0.7631579,0.7631579,0.8157895,0.8157895,0.8157895,0.8684211,0.8684211,0.8684211,0.92105263,0.92105263,0.92105263,0.97368425,0.97368425,0.9736843,0.02631579,0.026315793,0.02631579,0.078947365,0.078947365,0.07894737,0.13157895,0.13157894,0.13157895,0.18421052,0.18421052,0.18421051,0.23684211,0.23684211,0.23684211,0.28947368,0.28947365,0.28947368,0.34210524,0.3421052,0.34210524,0.39473683,0.39473683,0.39473683,0.4473684,0.4473684,0.4473684,0.5,0.5,0.5,0.5526316,0.5526316,0.5526316,0.6052632,0.6052632,0.6052632,0.65789473,0.65789473,0.65789473,0.71052635,0.71052635,0.71052635,0.7631579,0.7631579,0.7631579,0.8157895,0.8157895,0.8157895,0.8684211,0.8684211,0.8684211,0.92105263,0.92105263,0.92105263,0.97368425,0.97368425,0.9736843,0.02631579,0.026315793,0.02631579,0.078947365,0.078947365,0.07894737,0.13157895,0.13157894,0.13157895,0.18421052,0.18421052,0.18421051,0.23684211,0.23684211,0.23684211,0.28947368,0.28947365,0.28947368,0.34210524,0.3421052,0.34210524,0.39473683,0.39473683,0.39473683,0.4473684,0.4473684,0.4473684,0.5,0.5,0.5,0.5526316,0.5526316,0.5526316,0.6052632,0.6052632,0.6052632,0.65789473,0.65789473,0.65789473,0.71052635,0.71052635,0.71052635,0.7631579,0.7631579,0.7631579,0.8157895,0.8157895,0.8157895,0.8684211,0.8684211,0.8684211,0.92105263,0.92105263,0.92105263,0.97368425,0.97368425,0.9736843,0.02631579,0.026315793,0.02631579,0.078947365,0.078947365,0.07894737,0.13157895,0.13157894,0.13157895,0.18421052,0.18421052,0.18421051,0.23684211,0.23684211,0.23684211,0.28947368,0.28947365,0.28947368,0.34210524,0.3421052,0.34210524,0.39473683,0.39473683,0.39473683,0.4473684,0.4473684,0.4473684,0.5,0.5,0.5,0.5526316,0.5526316,0.5526316,0.6052632,0.6052632,0.6052632,0.65789473,0.65789473,0.65789473,0.71052635,0.71052635,0.71052635,0.7631579,0.7631579,0.7631579,0.8157895,0.8157895,0.8157895,0.8684211,0.8684211,0.8684211,0.92105263,0.92105263,0.92105263,0.97368425,0.97368425,0.9736843,0.02631579,0.026315793,0.02631579,0.078947365,0.078947365,0.07894737,0.13157895,0.13157894,0.13157895,0.18421052,0.18421052,0.18421051,0.23684211,0.23684211,0.23684211,0.28947368,0.28947365,0.28947368,0.34210524,0.3421052,0.34210524,0.39473683,0.39473683,0.39473683,0.4473684,0.4473684,0.4473684,0.5,0.5,0.5,0.5526316,0.5526316,0.5526316,0.6052632,0.6052632,0.6052632,0.65789473,0.65789473,0.65789473,0.71052635,0.71052635,0.71052635,0.7631579,0.7631579,0.7631579,0.8157895,0.8157895,0.8157895,0.8684211,0.8684211,0.8684211,0.92105263,0.92105263,0.92105263,0.97368425,0.97368425,0.9736843,0.02631579,0.026315793,0.02631579,0.078947365,0.078947365,0.07894737,0.13157895,0.13157894,0.13157895,0.18421052,0.18421052,0.18421051,0.23684211,0.23684211,0.23684211,0.28947368,0.28947365,0.28947368,0.34210524,0.3421052,0.34210524,0.39473683,0.39473683,0.39473683,0.4473684,0.4473684,0.4473684,0.5,0.5,0.5,0.5526316,0.5526316,0.5526316,0.6052632,0.6052632,0.6052632,0.65789473,0.65789473,0.65789473,0.71052635,0.71052635,0.71052635,0.7631579,0.7631579,0.7631579,0.8157895,0.8157895,0.8157895,0.8684211,0.8684211,0.8684211,0.92105263,0.92105263,0.92105263,0.97368425,0.97368425,0.9736843,0.02631579,0.026315793,0.02631579,0.078947365,0.078947365,0.07894737,0.13157895,0.13157894,0.13157895,0.18421052,0.18421052,0.18421051,0.23684211,0.23684211,0.23684211,0.28947368,0.28947365,0.28947368,0.34210524,0.3421052,0.34210524,0.39473683,0.39473683,0.39473683,0.4473684,0.4473684,0.4473684,0.5,0.5,0.5,0.5526316,0.5526316,0.5526316,0.6052632,0.6052632,0.6052632,0.65789473,0.65789473,0.65789473,0.71052635,0.71052635,0.71052635,0.7631579,0.7631579,0.7631579,0.8157895,0.8157895,0.8157895,0.8684211,0.8684211,0.8684211,0.92105263,0.92105263,0.92105263,0.97368425,0.97368425,0.9736843,0.02631579,0.026315793,0.02631579,0.078947365,0.078947365,0.07894737,0.13157895,0.13157894,0.13157895,0.18421052,0.18421052,0.18421051,0.23684211,0.23684211,0.23684211,0.28947368,0.28947365,0.28947368,0.34210524,0.3421052,0.34210524,0.39473683,0.39473683,0.39473683,0.4473684,0.4473684,0.4473684,0.5,0.5,0.5,0.5526316,0.5526316,0.5526316,0.6052632,0.6052632,0.6052632,0.65789473,0.65789473,0.65789473,0.71052635,0.71052635,0.71052635,0.7631579,0.7631579,0.7631579,0.8157895,0.8157895,0.8157895,0.8684211,0.8684211,0.8684211,0.92105263,0.92105263,0.92105263,0.97368425,0.97368425,0.9736843,0.02631579,0.026315793,0.02631579,0.078947365,0.078947365,0.07894737,0.13157895,0.13157894,0.13157895,0.18421052,0.18421052,0.18421051,0.23684211,0.23684211,0.23684211,0.28947368,0.28947365,0.28947368,0.34210524,0.3421052,0.34210524,0.39473683,0.39473683,0.39473683,0.4473684,0.4473684,0.4473684,0.5,0.5,0.5,0.5526316,0.5526316,0.5526316,0.6052632,0.6052632,0.6052632,0.65789473,0.65789473,0.65789473,0.71052635,0.71052635,0.71052635,0.7631579,0.7631579,0.7631579,0.8157895,0.8157895,0.8157895,0.8684211,0.8684211,0.8684211,0.92105263,0.92105263,0.92105263,0.97368425,0.97368425,0.9736843,0.02631579,0.026315793,0.02631579,0.078947365,0.078947365,0.07894737,0.13157895,0.13157894,0.13157895,0.18421052,0.18421052,0.18421051,0.23684211,0.23684211,0.23684211,0.28947368,0.28947365,0.28947368,0.34210524,0.3421052,0.34210524,0.39473683,0.39473683,0.39473683,0.4473684,0.4473684,0.4473684,0.5,0.5,0.5,0.5526316,0.5526316,0.5526316,0.6052632,0.6052632,0.6052632,0.65789473,0.65789473,0.65789473,0.71052635,0.71052635,0.71052635,0.7631579,0.7631579,0.7631579,0.8157895,0.8157895,0.8157895,0.8684211,0.8684211,0.8684211,0.92105263,0.92105263,0.92105263,0.97368425,0.97368425,0.9736843,0.02631579,0.026315793,0.02631579,0.078947365,0.078947365,0.07894737,0.13157895,0.13157894,0.13157895,0.18421052,0.18421052,0.18421051,0.23684211,0.23684211,0.23684211,0.28947368,0.28947365,0.28947368,0.34210524,0.3421052,0.34210524,0.39473683,0.39473683,0.39473683,0.4473684,0.4473684,0.4473684,0.5,0.5,0.5,0.5526316,0.5526316,0.5526316,0.6052632,0.6052632,0.6052632,0.65789473,0.65789473,0.65789473,0.71052635,0.71052635,0.71052635,0.7631579,0.7631579,0.7631579,0.8157895,0.8157895,0.8157895,0.8684211,0.8684211,0.8684211,0.92105263,0.92105263,0.92105263,0.97368425,0.97368425,0.9736843,0.02631579,0.026315793,0.02631579,0.078947365,0.078947365,0.07894737,0.13157895,0.13157894,0.13157895,0.18421052,0.18421052,0.18421051,0.23684211,0.23684211,0.23684211,0.28947368,0.28947365,0.28947368,0.34210524,0.3421052,0.34210524,0.39473683,0.39473683,0.39473683,0.4473684,0.4473684,0.4473684,0.5,0.5,0.5,0.5526316,0.5526316,0.5526316,0.6052632,0.6052632,0.6052632,0.65789473,0.65789473,0.65789473,0.71052635,0.71052635,0.71052635,0.7631579,0.7631579,0.7631579,0.8157895,0.8157895,0.8157895,0.8684211,0.8684211,0.8684211,0.92105263,0.92105263,0.92105263,0.97368425,0.97368425,0.9736843,0.02631579,0.026315793,0.02631579,0.078947365,0.078947365,0.07894737,0.13157895,0.13157894,0.13157895,0.18421052,0.18421052,0.18421051,0.23684211,0.23684211,0.23684211,0.28947368,0.28947365,0.28947368,0.34210524,0.3421052,0.34210524,0.39473683,0.39473683,0.39473683,0.4473684,0.4473684,0.4473684,0.5,0.5,0.5,0.5526316,0.5526316,0.5526316,0.6052632,0.6052632,0.6052632,0.65789473,0.65789473,0.65789473,0.71052635,0.71052635,0.71052635,0.7631579,0.7631579,0.7631579,0.8157895,0.8157895,0.8157895,0.8684211,0.8684211,0.8684211,0.92105263,0.92105263,0.92105263,0.97368425,0.97368425,0.9736843,0.02631579,0.026315793,0.02631579,0.078947365,0.078947365,0.07894737,0.13157895,0.13157894,0.13157895,0.18421052,0.18421052,0.18421051,0.23684211,0.23684211,0.23684211,0.28947368,0.28947365,0.28947368,0.34210524,0.3421052,0.34210524,0.39473683,0.39473683,0.39473683,0.4473684,0.4473684,0.4473684,0.5,0.5,0.5,0.5526316,0.5526316,0.5526316,0.6052632,0.6052632,0.6052632,0.65789473,0.65789473,0.65789473,0.71052635,0.71052635,0.71052635,0.7631579,0.7631579,0.7631579,0.8157895,0.8157895,0.8157895,0.8684211,0.8684211,0.8684211,0.92105263,0.92105263,0.92105263,0.97368425,0.97368425,0.9736843,0.02631579,0.026315793,0.02631579,0.078947365,0.078947365,0.07894737,0.13157895,0.13157894,0.13157895,0.18421052,0.18421052,0.18421051,0.23684211,0.23684211,0.23684211,0.28947368,0.28947365,0.28947368,0.34210524,0.3421052,0.34210524,0.39473683,0.39473683,0.39473683,0.4473684,0.4473684,0.4473684,0.5,0.5,0.5,0.5526316,0.5526316,0.5526316,0.6052632,0.6052632,0.6052632,0.65789473,0.65789473,0.65789473,0.71052635,0.71052635,0.71052635,0.7631579,0.7631579,0.7631579,0.8157895,0.8157895,0.8157895,0.8684211,0.8684211,0.8684211,0.92105263,0.92105263,0.92105263,0.97368425,0.97368425,0.9736843,0.02631579,0.026315793,0.02631579,0.078947365,0.078947365,0.07894737,0.13157895,0.13157894,0.13157895,0.18421052,0.18421052,0.18421051,0.23684211,0.23684211,0.23684211,0.28947368,0.28947365,0.28947368,0.34210524,0.3421052,0.34210524,0.39473683,0.39473683,0.39473683,0.4473684,0.4473684,0.4473684,0.5,0.5,0.5,0.5526316,0.5526316,0.5526316,0.6052632,0.6052632,0.6052632,0.65789473,0.65789473,0.65789473,0.71052635,0.71052635,0.71052635,0.7631579,0.7631579,0.7631579,0.8157895,0.8157895,0.8157895,0.8684211,0.8684211,0.8684211,0.92105263,0.92105263,0.92105263,0.97368425,0.97368425,0.9736843,0.02631579,0.026315793,0.02631579,0.078947365,0.078947365,0.07894737,0.13157895,0.13157894,0.13157895,0.18421052,0.18421052,0.18421051,0.23684211,0.23684211,0.23684211,0.28947368,0.28947365,0.28947368,0.34210524,0.3421052,0.34210524,0.39473683,0.39473683,0.39473683,0.4473684,0.4473684,0.4473684,0.5,0.5,0.5,0.5526316,0.5526316,0.5526316,0.6052632,0.6052632,0.6052632,0.65789473,0.65789473,0.65789473,0.71052635,0.71052635,0.71052635,0.7631579,0.7631579,0.7631579,0.8157895,0.8157895,0.8157895,0.8684211,0.8684211,0.8684211,0.92105263,0.92105263,0.92105263,0.97368425,0.97368425,0.9736843,0.02631579,0.026315793,0.02631579,0.078947365,0.078947365,0.07894737,0.13157895,0.13157894,0.13157895,0.18421052,0.18421052,0.18421051,0.23684211,0.23684211,0.23684211,0.28947368,0.28947365,0.28947368,0.34210524,0.3421052,0.34210524,0.39473683,0.39473683,0.39473683,0.4473684,0.4473684,0.4473684,0.5,0.5,0.5,0.5526316,0.5526316,0.5526316,0.6052632,0.6052632,0.6052632,0.65789473,0.65789473,0.65789473,0.71052635,0.71052635,0.71052635,0.7631579,0.7631579,0.7631579,0.8157895,0.8157895,0.8157895,0.8684211,0.8684211,0.8684211,0.92105263,0.92105263,0.92105263,0.97368425,0.97368425,0.9736843,0.049999997,0.049999997,0.050000004,0.050000012,0.05,0.049999997,0.15,0.14999999,0.15,0.15,0.15,0.15,0.25,0.25,0.25,0.25,0.25,0.25,0.35000002,0.35000002,0.35000002,0.35000002,0.35000002,0.35000002,0.45000002,0.45,0.45000002,0.45000002,0.45,0.45000002,0.55,0.55,0.55,0.55,0.55,0.55,0.65000004,0.65000004,0.65000004,0.65000004,0.65000004,0.65000004,0.75,0.75,0.75,0.75,0.75,0.75,0.85,0.85,0.85,0.85,0.85,0.85,0.95000005,0.95000005,0.95000005,0.95000005,0.95000005,0.95000005,0.049999997,0.049999997,0.050000004,0.050000012,0.05,0.049999997,0.15,0.14999999,0.15,0.15,0.15,0.15,0.25,0.25,0.25,0.25,0.25,0.25,0.35000002,0.35000002,0.35000002,0.35000002,0.35000002,0.35000002,0.45000002,0.45,0.45000002,0.45000002,0.45,0.45000002,0.55,0.55,0.55,0.55,0.55,0.55,0.65000004,0.65000004,0.65000004,0.65000004,0.65000004,0.65000004,0.75,0.75,0.75,0.75,0.75,0.75,0.85,0.85,0.85,0.85,0.85,0.85,0.95000005,0.95000005,0.95000005,0.95000005,0.95000005,0.95000005,0.049999997,0.049999997,0.050000004,0.050000012,0.05,0.049999997,0.15,0.14999999,0.15,0.15,0.15,0.15,0.25,0.25,0.25,0.25,0.25,0.25,0.35000002,0.35000002,0.35000002,0.35000002,0.35000002,0.35000002,0.45000002,0.45,0.45000002,0.45000002,0.45,0.45000002,0.55,0.55,0.55,0.55,0.55,0.55,0.65000004,0.65000004,0.65000004,0.65000004,0.65000004,0.65000004,0.75,0.75,0.75,0.75,0.75,0.75,0.85,0.85,0.85,0.85,0.85,0.85,0.95000005,0.95000005,0.95000005,0.95000005,0.95000005,0.95000005,0.049999997,0.049999997,0.050000004,0.050000012,0.05,0.049999997,0.15,0.14999999,0.15,0.15,0.15,0.15,0.25,0.25,0.25,0.25,0.25,0.25,0.35000002,0.35000002,0.35000002,0.35000002,0.35000002,0.35000002,0.45000002,0.45,0.45000002,0.45000002,0.45,0.45000002,0.55,0.55,0.55,0.55,0.55,0.55,0.65000004,0.65000004,0.65000004,0.65000004,0.65000004,0.65000004,0.75,0.75,0.75,0.75,0.75,0.75,0.85,0.85,0.85,0.85,0.85,0.85,0.95000005,0.95000005,0.95000005,0.95000005,0.95000005,0.95000005,0.049999997,0.049999997,0.050000004,0.050000012,0.05,0.049999997,0.15,0.14999999,0.15,0.15,0.15,0.15,0.25,0.25,0.25,0.25,0.25,0.25,0.35000002,0.35000002,0.35000002,0.35000002,0.35000002,0.35000002,0.45000002,0.45,0.45000002,0.45000002,0.45,0.45000002,0.55,0.55,0.55,0.55,0.55,0.55,0.65000004,0.65000004,0.65000004,0.65000004,0.65000004,0.65000004,0.75,0.75,0.75,0.75,0.75,0.75,0.85,0.85,0.85,0.85,0.85,0.85,0.95000005,0.95000005,0.95000005,0.95000005,0.95000005,0.95000005,0.049999997,0.049999997,0.050000004,0.050000012,0.05,0.049999997,0.15,0.14999999,0.15,0.15,0.15,0.15,0.25,0.25,0.25,0.25,0.25,0.25,0.35000002,0.35000002,0.35000002,0.35000002,0.35000002,0.35000002,0.45000002,0.45,0.45000002,0.45000002,0.45,0.45000002,0.55,0.55,0.55,0.55,0.55,0.55,0.65000004,0.65000004,0.65000004,0.65000004,0.65000004,0.65000004,0.75,0.75,0.75,0.75,0.75,0.75,0.85,0.85,0.85,0.85,0.85,0.85,0.95000005,0.95000005,0.95000005,0.95000005,0.95000005,0.95000005,0.049999997,0.049999997,0.050000004,0.050000012,0.05,0.049999997,0.15,0.14999999,0.15,0.15,0.15,0.15,0.25,0.25,0.25,0.25,0.25,0.25,0.35000002,0.35000002,0.35000002,0.35000002,0.35000002,0.35000002,0.45000002,0.45,0.45000002,0.45000002,0.45,0.45000002,0.55,0.55,0.55,0.55,0.55,0.55,0.65000004,0.65000004,0.65000004,0.65000004,0.65000004,0.65000004,0.75,0.75,0.75,0.75,0.75,0.75,0.85,0.85,0.85,0.85,0.85,0.85,0.95000005,0.95000005,0.95000005,0.95000005,0.95000005,0.95000005,0.049999997,0.049999997,0.050000004,0.050000012,0.05,0.049999997,0.15,0.14999999,0.15,0.15,0.15,0.15,0.25,0.25,0.25,0.25,0.25,0.25,0.35000002,0.35000002,0.35000002,0.35000002,0.35000002,0.35000002,0.45000002,0.45,0.45000002,0.45000002,0.45,0.45000002,0.55,0.55,0.55,0.55,0.55,0.55,0.65000004,0.65000004,0.65000004,0.65000004,0.65000004,0.65000004,0.75,0.75,0.75,0.75,0.75,0.75,0.85,0.85,0.85,0.85,0.85,0.85,0.95000005,0.95000005,0.95000005,0.95000005,0.95000005,0.95000005,0.049999997,0.049999997,0.050000004,0.050000012,0.05,0.049999997,0.15,0.14999999,0.15,0.15,0.15,0.15,0.25,0.25,0.25,0.25,0.25,0.25,0.35000002,0.35000002,0.35000002,0.35000002,0.35000002,0.35000002,0.45000002,0.45,0.45000002,0.45000002,0.45,0.45000002,0.55,0.55,0.55,0.55,0.55,0.55,0.65000004,0.65000004,0.65000004,0.65000004,0.65000004,0.65000004,0.75,0.75,0.75,0.75,0.75,0.75,0.85,0.85,0.85,0.85,0.85,0.85,0.95000005,0.95000005,0.95000005,0.95000005,0.95000005,0.95000005,0.049999997,0.049999997,0.050000004,0.050000012,0.05,0.049999997,0.15,0.14999999,0.15,0.15,0.15,0.15,0.25,0.25,0.25,0.25,0.25,0.25,0.35000002,0.35000002,0.35000002,0.35000002,0.35000002,0.35000002,0.45000002,0.45,0.45000002,0.45000002,0.45,0.45000002,0.55,0.55,0.55,0.55,0.55,0.55,0.65000004,0.65000004,0.65000004,0.65000004,0.65000004,0.65000004,0.75,0.75,0.75,0.75,0.75,0.75,0.85,0.85,0.85,0.85,0.85,0.85,0.95000005,0.95000005,0.95000005,0.95000005,0.95000005,0.95000005,0.10000001,0.099999994,0.1,0.099999994,0.1,0.099999994,0.3,0.3,0.3,0.29999998,0.3,0.30000004,0.5,0.5,0.5,0.5,0.5,0.49999997,0.70000005,0.70000005,0.70000005,0.70000005,0.70000005,0.70000005,0.9,0.90000004,0.90000004,0.9,0.90000004,0.90000004,0.10000001,0.099999994,0.1,0.099999994,0.1,0.099999994,0.3,0.3,0.3,0.29999998,0.3,0.30000004,0.5,0.5,0.5,0.5,0.5,0.49999997,0.70000005,0.70000005,0.70000005,0.70000005,0.70000005,0.70000005,0.9,0.90000004,0.90000004,0.9,0.90000004,0.90000004,0.10000001,0.099999994,0.1,0.099999994,0.1,0.099999994,0.3,0.3,0.3,0.29999998,0.3,0.30000004,0.5,0.5,0.5,0.5,0.5,0.49999997,0.70000005,0.70000005,0.70000005,0.70000005,0.70000005,0.70000005,0.9,0.90000004,0.90000004,0.9,0.90000004,0.90000004,0.10000001,0.099999994,0.1,0.099999994,0.1,0.099999994,0.3,0.3,0.3,0.29999998,0.3,0.30000004,0.5,0.5,0.5,0.5,0.5,0.49999997,0.70000005,0.70000005,0.70000005,0.70000005,0.70000005,0.70000005,0.9,0.90000004,0.90000004,0.9,0.90000004,0.90000004,0.10000001,0.099999994,0.1,0.099999994,0.1,0.099999994,0.3,0.3,0.3,0.29999998,0.3,0.30000004,0.5,0.5,0.5,0.5,0.5,0.49999997,0.70000005,0.70000005,0.70000005,0.70000005,0.70000005,0.70000005,0.9,0.90000004,0.90000004,0.9,0.90000004,0.90000004,0.16666667,0.16666669,0.16666667,0.16666669,0.16666667,0.16666667,0.49999997,0.5,0.5,0.50000006,0.5,0.5,0.8333334,0.8333334,0.8333334,0.8333333,0.8333334,0.8333334,0.16666667,0.16666669,0.16666667,0.16666669,0.16666667,0.16666667,0.49999997,0.5,0.5,0.50000006,0.5,0.5,0.8333334,0.8333334,0.8333334,0.8333333,0.8333334,0.8333334,0.16666667,0.16666669,0.16666667,0.16666669,0.16666667,0.16666667,0.49999997,0.5,0.5,0.50000006,0.5,0.5,0.8333334,0.8333334,0.8333334,0.8333333,0.8333334,0.8333334,0.25,0.25,0.25,0.25,0.25,0.25,0.75,0.75,0.75,0.75,0.75,0.75,0.25,0.25,0.25,0.25,0.25,0.25,0.75,0.75,0.75,0.75,0.75,0.75,0.5,0.5,0.5,0.5,0.5,0.5,
0.1,0.14142136,0.28284273,0.1,0.14142136,0.28284273,0.1,0.14142136,0.28284273,0.1,0.14142136,0.28284273,0.1,0.14142136,0.28284273,0.1,0.14142136,0.28284273,0.1,0.14142136,0.28284273,0.1,0.14142136,0.28284273,0.1,0.14142136,0.28284273,0.1,0.14142136,0.28284273,0.1,0.14142136,0.28284273,0.1,0.14142136,0.28284273,0.1,0.14142136,0.28284273,0.1,0.14142136,0.28284273,0.1,0.14142136,0.28284273,0.1,0.14142136,0.28284273,0.1,0.14142136,0.28284273,0.1,0.14142136,0.28284273,0.1,0.14142136,0.28284273,0.099999994,0.14142138,0.28284273,0.099999994,0.14142138,0.28284273,0.099999994,0.14142138,0.28284273,0.099999994,0.14142138,0.28284273,0.099999994,0.14142138,0.28284273,0.099999994,0.14142138,0.28284273,0.099999994,0.14142138,0.28284273,0.099999994,0.14142138,0.28284273,0.099999994,0.14142138,0.28284273,0.099999994,0.14142138,0.28284273,0.099999994,0.14142138,0.28284273,0.099999994,0.14142138,0.28284273,0.099999994,0.14142138,0.28284273,0.099999994,0.14142138,0.28284273,0.099999994,0.14142138,0.28284273,0.099999994,0.14142138,0.28284273,0.099999994,0.14142138,0.28284273,0.099999994,0.14142138,0.28284273,0.099999994,0.14142138,0.28284273,0.099999994,0.14142138,0.2828427,0.099999994,0.14142138,0.2828427,0.099999994,0.14142138,0.2828427,0.099999994,0.14142138,0.2828427,0.099999994,0.14142138,0.2828427,0.099999994,0.14142138,0.2828427,0.099999994,0.14142138,0.2828427,0.099999994,0.14142138,0.2828427,0.099999994,0.14142138,0.2828427,0.099999994,0.14142138,0.2828427,0.099999994,0.14142138,0.2828427,0.099999994,0.14142138,0.2828427,0.099999994,0.14142138,0.2828427,0.099999994,0.14142138,0.2828427,0.099999994,0.14142138,0.2828427,0.099999994,0.14142138,0.2828427,0.099999994,0.14142138,0.2828427,0.099999994,0.14142138,0.2828427,0.099999994,0.14142138,0.2828427,0.099999994,0.14142135,0.28284273,0.099999994,0.14142135,0.28284273,0.099999994,0.14142135,0.28284273,0.099999994,0.14142135,0.28284273,0.099999994,0.14142135,0.28284273,0.099999994,0.14142135,0.28284273,0.099999994,0.14142135,0.28284273,0.099999994,0.14142135,0.28284273,0.099999994,0.14142135,0.28284273,0.099999994,0.14142135,0.28284273,0.099999994,0.14142135,0.28284273,0.099999994,0.14142135,0.28284273,0.099999994,0.14142135,0.28284273,0.099999994,0.14142135,0.28284273,0.099999994,0.14142135,0.28284273,0.099999994,0.14142135,0.28284273,0.099999994,0.14142135,0.28284273,0.099999994,0.14142135,0.28284273,0.099999994,0.14142135,0.28284273,0.099999994,0.14142135,0.28284273,0.099999994,0.14142135,0.28284273,0.099999994,0.14142135,0.28284273,0.099999994,0.14142135,0.28284273,0.099999994,0.14142135,0.28284273,0.099999994,0.14142135,0.28284273,0.099999994,0.14142135,0.28284273,0.099999994,0.14142135,0.28284273,0.099999994,0.14142135,0.28284273,0.099999994,0.14142135,0.28284273,0.099999994,0.14142135,0.28284273,0.099999994,0.14142135,0.28284273,0.099999994,0.14142135,0.28284273,0.099999994,0.14142135,0.28284273,0.099999994,0.14142135,0.28284273,0.099999994,0.14142135,0.28284273,0.099999994,0.14142135,0.28284273,0.099999994,0.14142135,0.28284273,0.099999994,0.14142135,0.28284273,0.10000001,0.14142138,0.2828427,0.10000001,0.14142138,0.2828427,0.10000001,0.14142138,0.2828427,0.10000001,0.14142138,0.2828427,0.10000001,0.14142138,0.2828427,0.10000001,0.14142138,0.2828427,0.10000001,0.14142138,0.2828427,0.10000001,0.14142138,0.2828427,0.10000001,0.14142138,0.2828427,0.10000001,0.14142138,0.2828427,0.10000001,0.14142138,0.2828427,0.10000001,0.14142138,0.2828427,0.10000001,0.14142138,0.2828427,0.10000001,0.14142138,0.2828427,0.10000001,0.14142138,0.2828427,0.10000001,0.14142138,0.2828427,0.10000001,0.14142138,0.2828427,0.10000001,0.14142138,0.2828427,0.10000001,0.14142138,0.2828427,0.100000024,0.14142138,0.2828427,0.100000024,0.14142138,0.2828427,0.100000024,0.14142138,0.2828427,0.100000024,0.14142138,0.2828427,0.100000024,0.14142138,0.2828427,0.100000024,0.14142138,0.2828427,0.100000024,0.14142138,0.2828427,0.100000024,0.14142138,0.2828427,0.100000024,0.14142138,0.2828427,0.100000024,0.14142138,0.2828427,0.100000024,0.14142138,0.2828427,0.100000024,0.14142138,0.2828427,0.100000024,0.14142138,0.2828427,0.100000024,0.14142138,0.2828427,0.100000024,0.14142138,0.2828427,0.100000024,0.14142138,0.2828427,0.100000024,0.14142138,0.2828427,0.100000024,0.14142138,0.2828427,0.100000024,0.14142138,0.2828427,0.100000024,0.14142138,0.28284276,0.100000024,0.14142138,0.28284276,0.100000024,0.14142138,0.28284276,0.100000024,0.14142138,0.28284276,0.100000024,0.14142138,0.28284276,0.100000024,0.14142138,0.28284276,0.100000024,0.14142138,0.28284276,0.100000024,0.14142138,0.28284276,0.100000024,0.14142138,0.28284276,0.100000024,0.14142138,0.28284276,0.100000024,0.14142138,0.28284276,0.100000024,0.14142138,0.28284276,0.100000024,0.14142138,0.28284276,0.100000024,0.14142138,0.28284276,0.100000024,0.14142138,0.28284276,0.100000024,0.14142138,0.28284276,0.100000024,0.14142138,0.28284276,0.100000024,0.14142138,0.28284276,0.100000024,0.14142138,0.28284276,0.100000024,0.14142138,0.2828427,0.100000024,0.14142138,0.2828427,0.100000024,0.14142138,0.2828427,0.100000024,0.14142138,0.2828427,0.100000024,0.14142138,0.2828427,0.100000024,0.14142138,0.2828427,0.100000024,0.14142138,0.2828427,0.100000024,0.14142138,0.2828427,0.100000024,0.14142138,0.2828427,0.100000024,0.14142138,0.2828427,0.100000024,0.14142138,0.2828427,0.100000024,0.14142138,0.2828427,0.100000024,0.14142138,0.2828427,0.100000024,0.14142138,0.2828427,0.100000024,0.14142138,0.2828427,0.100000024,0.14142138,0.2828427,0.100000024,0.14142138,0.2828427,0.100000024,0.14142138,0.2828427,0.100000024,0.14142138,0.2828427,0.100000024,0.14142135,0.28284276,0.100000024,0.14142135,0.28284276,0.100000024,0.14142135,0.28284276,0.100000024,0.14142135,0.28284276,0.100000024,0.14142135,0.28284276,0.100000024,0.14142135,0.28284276,0.100000024,0.14142135,0.28284276,0.100000024,0.14142135,0.28284276,0.100000024,0.14142135,0.28284276,0.100000024,0.14142135,0.28284276,0.100000024,0.14142135,0.28284276,0.100000024,0.14142135,0.28284276,0.100000024,0.14142135,0.28284276,0.100000024,0.14142135,0.28284276,0.100000024,0.14142135,0.28284276,0.100000024,0.14142135,0.28284276,0.100000024,0.14142135,0.28284276,0.100000024,0.14142135,0.28284276,0.100000024,0.14142135,0.28284276,0.100000024,0.14142135,0.28284276,0.100000024,0.14142135,0.28284276,0.100000024,0.14142135,0.28284276,0.100000024,0.14142135,0.28284276,0.100000024,0.14142135,0.28284276,0.100000024,0.14142135,0.28284276,0.100000024,0.14142135,0.28284276,0.100000024,0.14142135,0.28284276,0.100000024,0.14142135,0.28284276,0.100000024,0.14142135,0.28284276,0.100000024,0.14142135,0.28284276,0.100000024,0.14142135,0.28284276,0.100000024,0.14142135,0.28284276,0.100000024,0.14142135,0.28284276,0.100000024,0.14142135,0.28284276,0.100000024,0.14142135,0.28284276,0.100000024,0.14142135,0.28284276,0.100000024,0.14142135,0.28284276,0.100000024,0.14142135,0.28284276,0.100000024,0.14142132,0.28284276,0.100000024,0.14142132,0.28284276,0.100000024,0.14142132,0.28284276,0.100000024,0.14142132,0.28284276,0.100000024,0.14142132,0.28284276,0.100000024,0.14142132,0.28284276,0.100000024,0.14142132,0.28284276,0.100000024,0.14142132,0.28284276,0.100000024,0.14142132,0.28284276,0.100000024,0.14142132,0.28284276,0.100000024,0.14142132,0.28284276,0.100000024,0.14142132,0.28284276,0.100000024,0.14142132,0.28284276,0.100000024,0.14142132,0.28284276,0.100000024,0.14142132,0.28284276,0.100000024,0.14142132,0.28284276,0.100000024,0.14142132,0.28284276,0.100000024,0.14142132,0.28284276,0.100000024,0.14142132,0.28284276,0.100000024,0.14142132,0.28284276,0.100000024,0.14142132,0.28284276,0.100000024,0.14142132,0.28284276,0.100000024,0.14142132,0.28284276,0.100000024,0.14142132,0.28284276,0.100000024,0.14142132,0.28284276,0.100000024,0.14142132,0.28284276,0.100000024,0.14142132,0.28284276,0.100000024,0.14142132,0.28284276,0.100000024,0.14142132,0.28284276,0.100000024,0.14142132,0.28284276,0.100000024,0.14142132,0.28284276,0.100000024,0.14142132,0.28284276,0.100000024,0.14142132,0.28284276,0.100000024,0.14142132,0.28284276,0.100000024,0.14142132,0.28284276,0.100000024,0.14142132,0.28284276,0.100000024,0.14142132,0.28284276,0.100000024,0.14142132,0.28284276,0.100000024,0.14142132,0.28284276,0.100000024,0.14142132,0.28284276,0.100000024,0.14142132,0.28284276,0.100000024,0.14142132,0.28284276,0.100000024,0.14142132,0.28284276,0.100000024,0.14142132,0.28284276,0.100000024,0.14142132,0.28284276,0.100000024,0.14142132,0.28284276,0.100000024,0.14142132,0.28284276,0.100000024,0.14142132,0.28284276,0.100000024,0.14142132,0.28284276,0.100000024,0.14142132,0.28284276,0.100000024,0.14142132,0.28284276,0.100000024,0.14142132,0.28284276,0.100000024,0.14142132,0.28284276,0.100000024,0.14142132,0.28284276,0.100000024,0.14142132,0.28284276,0.100000024,0.14142132,0.28284276,0.100000024,0.14142132,0.28284276,0.100000024,0.14142132,0.28284276,0.100000024,0.14142132,0.28284276,0.100000024,0.14142132,0.28284276,0.100000024,0.14142132,0.28284276,0.100000024,0.14142132,0.28284276,0.100000024,0.14142132,0.28284276,0.100000024,0.14142132,0.28284276,0.100000024,0.14142132,0.28284276,0.100000024,0.14142132,0.28284276,0.100000024,0.14142132,0.28284276,0.100000024,0.14142132,0.28284276,0.100000024,0.14142132,0.28284276,0.100000024,0.14142132,0.28284276,0.100000024,0.14142132,0.28284276,0.100000024,0.14142132,0.28284276,0.100000024,0.14142132,0.28284276,0.100000024,0.14142132,0.28284276,0.100000024,0.14142132,0.28284276,0.100000024,0.14142132,0.28284276,0.100000024,0.14142132,0.28284276,0.100000024,0.14142132,0.28284276,0.100000024,0.14142132,0.28284276,0.100000024,0.14142132,0.28284276,0.100000024,0.14142132,0.28284276,0.100000024,0.14142132,0.28284276,0.100000024,0.14142132,0.28284276,0.100000024,0.14142132,0.28284276,0.100000024,0.14142132,0.28284276,0.100000024,0.14142132,0.28284276,0.100000024,0.14142132,0.28284276,0.100000024,0.14142132,0.28284276,0.100000024,0.14142132,0.28284276,0.100000024,0.14142132,0.28284276,0.100000024,0.14142132,0.28284276,0.100000024,0.14142132,0.28284276,0.100000024,0.14142132,0.28284276,0.100000024,0.14142132,0.28284276,0.100000024,0.14142132,0.28284276,0.100000024,0.14142132,0.2828427,0.100000024,0.14142132,0.2828427,0.100000024,0.14142132,0.2828427,0.100000024,0.14142132,0.2828427,0.100000024,0.14142132,0.2828427,0.100000024,0.14142132,0.2828427,0.100000024,0.14142132,0.2828427,0.100000024,0.14142132,0.2828427,0.100000024,0.14142132,0.2828427,0.100000024,0.14142132,0.2828427,0.100000024,0.14142132,0.2828427,0.100000024,0.14142132,0.2828427,0.100000024,0.14142132,0.2828427,0.100000024,0.14142132,0.2828427,0.100000024,0.14142132,0.2828427,0.100000024,0.14142132,0.2828427,0.100000024,0.14142132,0.2828427,0.100000024,0.14142132,0.2828427,0.100000024,0.14142132,0.2828427,0.100000024,0.14142132,0.28284276,0.100000024,0.14142132,0.28284276,0.100000024,0.14142132,0.28284276,0.100000024,0.14142132,0.28284276,0.100000024,0.14142132,0.28284276,0.100000024,0.14142132,0.28284276,0.100000024,0.14142132,0.28284276,0.100000024,0.14142132,0.28284276,0.100000024,0.14142132,0.28284276,0.100000024,0.14142132,0.28284276,0.100000024,0.14142132,0.28284276,0.100000024,0.14142132,0.28284276,0.100000024,0.14142132,0.28284276,0.100000024,0.14142132,0.28284276,0.100000024,0.14142132,0.28284276,0.100000024,0.14142132,0.28284276,0.100000024,0.14142132,0.28284276,0.100000024,0.14142132,0.28284276,0.100000024,0.14142132,0.28284276,0.100000024,0.14142138,0.28284276,0.100000024,0.14142138,0.28284276,0.100000024,0.14142138,0.28284276,0.100000024,0.14142138,0.28284276,0.100000024,0.14142138,0.28284276,0.100000024,0.14142138,0.28284276,0.100000024,0.14142138,0.28284276,0.100000024,0.14142138,0.28284276,0.100000024,0.14142138,0.28284276,0.100000024,0.14142138,0.28284276,0.100000024,0.14142138,0.28284276,0.100000024,0.14142138,0.28284276,0.100000024,0.14142138,0.28284276,0.100000024,0.14142138,0.28284276,0.100000024,0.14142138,0.28284276,0.100000024,0.14142138,0.28284276,0.100000024,0.14142138,0.28284276,0.100000024,0.14142138,0.28284276,0.100000024,0.14142138,0.28284276,0.35000002,0.2474874,0.4949748,0.20207259,0.6062481,0.41833,0.35000002,0.2474874,0.4949748,0.20207259,0.6062481,0.41833,0.35000002,0.2474874,0.4949748,0.20207259,0.6062481,0.41833,0.35000002,0.2474874,0.4949748,0.20207259,0.6062481,0.41833,0.35000002,0.2474874,0.4949748,0.20207259,0.6062481,0.41833,0.35000002,0.2474874,0.4949748,0.20207259,0.6062481,0.41833,0.35000002,0.2474874,0.4949748,0.20207259,0.6062481,0.41833,0.35000002,0.2474874,0.4949748,0.20207259,0.6062481,0.41833,0.35000002,0.2474874,0.4949748,0.20207259,0.6062481,0.41833,0.35000002,0.2474874,0.4949748,0.20207259,0.6062481,0.41833,0.35000002,0.24748738,0.4949748,0.20207258,0.6062481,0.41833,0.35000002,0.24748738,0.4949748,0.20207258,0.6062481,0.41833,0.35000002,0.24748738,0.4949748,0.20207258,0.6062481,0.41833,0.35000002,0.24748738,0.4949748,0.20207258,0.6062481,0.41833,0.35000002,0.24748738,0.4949748,0.20207258,0.6062481,0.41833,0.35000002,0.24748738,0.4949748,0.20207258,0.6062481,0.41833,0.35000002,0.24748738,0.4949748,0.20207258,0.6062481,0.41833,0.35000002,0.24748738,0.4949748,0.20207258,0.6062481,0.41833,0.35000002,0.24748738,0.4949748,0.20207258,0.6062481,0.41833,0.35000002,0.24748738,0.4949748,0.20207258,0.6062481,0.41833,0.35000002,0.24748741,0.4949748,0.2020726,0.60624814,0.41833,0.35000002,0.24748741,0.4949748,0.2020726,0.60624814,0.41833,0.35000002,0.24748741,0.4949748,0.2020726,0.60624814,0.41833,0.35000002,0.24748741,0.4949748,0.2020726,0.60624814,0.41833,0.35000002,0.24748741,0.4949748,0.2020726,0.60624814,0.41833,0.35000002,0.24748741,0.4949748,0.2020726,0.60624814,0.41833,0.35000002,0.24748741,0.4949748,0.2020726,0.60624814,0.41833,0.35000002,0.24748741,0.4949748,0.2020726,0.60624814,0.41833,0.35000002,0.24748741,0.4949748,0.2020726,0.60624814,0.41833,0.35000002,0.24748741,0.4949748,0.2020726,0.60624814,0.41833,0.35000002,0.24748741,0.49497482,0.2020726,0.60624814,0.41832998,0.35000002,0.24748741,0.49497482,0.2020726,0.60624814,0.41832998,0.35000002,0.24748741,0.49497482,0.2020726,0.60624814,0.41832998,0.35000002,0.24748741,0.49497482,0.2020726,0.60624814,0.41832998,0.35000002,0.24748741,0.49497482,0.2020726,0.60624814,0.41832998,0.35000002,0.24748741,0.49497482,0.2020726,0.60624814,0.41832998,0.35000002,0.24748741,0.49497482,0.2020726,0.60624814,0.41832998,0.35000002,0.24748741,0.49497482,0.2020726,0.60624814,0.41832998,0.35000002,0.24748741,0.49497482,0.2020726,0.60624814,0.41832998,0.35000002,0.24748741,0.49497482,0.2020726,0.60624814,0.41832998,0.35,0.24748737,0.4949748,0.20207256,0.6062481,0.41833,0.35,0.24748737,0.4949748,0.20207256,0.6062481,0.41833,0.35,0.24748737,0.4949748,0.20207256,0.6062481,0.41833,0.35,0.24748737,0.4949748,0.20207256,0.6062481,0.41833,0.35,0.24748737,0.4949748,0.20207256,0.6062481,0.41833,0.35,0.24748737,0.4949748,0.20207256,0.6062481,0.41833,0.35,0.24748737,0.4949748,0.20207256,0.6062481,0.41833,0.35,0.24748737,0.4949748,0.20207256,0.6062481,0.41833,0.35,0.24748737,0.4949748,0.20207256,0.6062481,0.41833,0.35,0.24748737,0.4949748,0.20207256,0.6062481,0.41833,0.35000002,0.24748743,0.49497476,0.20207262,0.606248,0.41833004,0.35000002,0.24748743,0.49497476,0.20207262,0.606248,0.41833004,0.35000002,0.24748743,0.49497476,0.20207262,0.606248,0.41833004,0.35000002,0.24748743,0.49497476,0.20207262,0.606248,0.41833004,0.35000002,0.24748743,0.49497476,0.20207262,0.606248,0.41833004,0.35000002,0.24748743,0.49497476,0.20207262,0.606248,0.41833004,0.35000002,0.24748743,0.49497476,0.20207262,0.606248,0.41833004,0.35000002,0.24748743,0.49497476,0.20207262,0.606248,0.41833004,0.35000002,0.24748743,0.49497476,0.20207262,0.606248,0.41833004,0.35000002,0.24748743,0.49497476,0.20207262,0.606248,0.41833004,0.35000002,0.24748743,0.49497476,0.20207262,0.606248,0.41833004,0.35000002,0.24748743,0.49497476,0.20207262,0.606248,0.41833004,0.35000002,0.24748743,0.49497476,0.20207262,0.606248,0.41833004,0.35000002,0.24748743,0.49497476,0.20207262,0.606248,0.41833004,0.35000002,0.24748743,0.49497476,0.20207262,0.606248,0.41833004,0.35000002,0.24748743,0.49497476,0.20207262,0.606248,0.41833004,0.35000002,0.24748743,0.49497476,0.20207262,0.606248,0.41833004,0.35000002,0.24748743,0.49497476,0.20207262,0.606248,0.41833004,0.35000002,0.24748743,0.49497476,0.20207262,0.606248,0.41833004,0.35000002,0.24748743,0.49497476,0.20207262,0.606248,0.41833004,0.35000002,0.24748743,0.49497485,0.20207262,0.60624814,0.41832995,0.35000002,0.24748743,0.49497485,0.20207262,0.60624814,0.41832995,0.35000002,0.24748743,0.49497485,0.20207262,0.60624814,0.41832995,0.35000002,0.24748743,0.49497485,0.20207262,0.60624814,0.41832995,0.35000002,0.24748743,0.49497485,0.20207262,0.60624814,0.41832995,0.35000002,0.24748743,0.49497485,0.20207262,0.60624814,0.41832995,0.35000002,0.24748743,0.49497485,0.20207262,0.60624814,0.41832995,0.35000002,0.24748743,0.49497485,0.20207262,0.60624814,0.41832995,0.35000002,0.24748743,0.49497485,0.20207262,0.60624814,0.41832995,0.35000002,0.24748743,0.49497485,0.20207262,0.60624814,0.41832995,0.35000008,0.24748743,0.49497485,0.20207262,0.60624814,0.41832995,0.35000008,0.24748743,0.49497485,0.20207262,0.60624814,0.41832995,0.35000008,0.24748743,0.49497485,0.20207262,0.60624814,0.41832995,0.35000008,0.24748743,0.49497485,0.20207262,0.60624814,0.41832995,0.35000008,0.24748743,0.49497485,0.20207262,0.60624814,0.41832995,0.35000008,0.24748743,0.49497485,0.20207262,0.60624814,0.41832995,0.35000008,0.24748743,0.49497485,0.20207262,0.60624814,0.41832995,0.35000008,0.24748743,0.49497485,0.20207262,0.60624814,0.41832995,0.35000008,0.24748743,0.49497485,0.20207262,0.60624814,0.41832995,0.35000008,0.24748743,0.49497485,0.20207262,0.60624814,0.41832995,0.34999996,0.24748737,0.49497485,0.20207262,0.60624814,0.41832995,0.34999996,0.24748737,0.49497485,0.20207262,0.60624814,0.41832995,0.34999996,0.24748737,0.49497485,0.20207262,0.60624814,0.41832995,0.34999996,0.24748737,0.49497485,0.20207262,0.60624814,0.41832995,0.34999996,0.24748737,0.49497485,0.20207262,0.60624814,0.41832995,0.34999996,0.24748737,0.49497485,0.20207262,0.60624814,0.41832995,0.34999996,0.24748737,0.49497485,0.20207262,0.60624814,0.41832995,0.34999996,0.24748737,0.49497485,0.20207262,0.60624814,0.41832995,0.34999996,0.24748737,0.49497485,0.20207262,0.60624814,0.41832995,0.34999996,0.24748737,0.49497485,0.20207262,0.60624814,0.41832995,0.50000006,0.3535534,0.7071068,0.28867513,0.8660687,0.57008773,0.50000006,0.3535534,0.7071068,0.28867513,0.8660687,0.57008773,0.50000006,0.3535534,0.7071068,0.28867513,0.8660687,0.57008773,0.50000006,0.3535534,0.7071068,0.28867513,0.8660687,0.57008773,0.50000006,0.3535534,0.7071068,0.28867513,0.8660687,0.57008773,0.5000001,0.3535534,0.7071068,0.28867513,0.8660687,0.5700878,0.5000001,0.3535534,0.7071068,0.28867513,0.8660687,0.5700878,0.5000001,0.3535534,0.7071068,0.28867513,0.8660687,0.5700878,0.5000001,0.3535534,0.7071068,0.28867513,0.8660687,0.5700878,0.5000001,0.3535534,0.7071068,0.28867513,0.8660687,0.5700878,0.5,0.3535534,0.7071068,0.2886751,0.8660687,0.5700877,0.5,0.3535534,0.7071068,0.2886751,0.8660687,0.5700877,0.5,0.3535534,0.7071068,0.2886751,0.8660687,0.5700877,0.5,0.3535534,0.7071068,0.2886751,0.8660687,0.5700877,0.5,0.3535534,0.7071068,0.2886751,0.8660687,0.5700877,0.5,0.3535534,0.7071068,0.28867507,0.8660688,0.5700877,0.5,0.3535534,0.7071068,0.28867507,0.8660688,0.5700877,0.5,0.3535534,0.7071068,0.28867507,0.8660688,0.5700877,0.5,0.3535534,0.7071068,0.28867507,0.8660688,0.5700877,0.5,0.3535534,0.7071068,0.28867507,0.8660688,0.5700877,0.5000001,0.3535534,0.70710677,0.2886752,0.8660687,0.5700878,0.5000001,0.3535534,0.70710677,0.2886752,0.8660687,0.5700878,0.5000001,0.3535534,0.70710677,0.2886752,0.8660687,0.5700878,0.5000001,0.3535534,0.70710677,0.2886752,0.8660687,0.5700878,0.5000001,0.3535534,0.70710677,0.2886752,0.8660687,0.5700878,0.65000004,0.45961943,0.91923887,0.37527767,1.1258893,0.7211102,0.65000004,0.45961943,0.91923887,0.37527767,1.1258893,0.7211102,0.65000004,0.45961943,0.91923887,0.37527767,1.1258893,0.7211102,0.6500001,0.4596194,0.9192388,0.37527764,1.1258893,0.7211102,0.6500001,0.4596194,0.9192388,0.37527764,1.1258893,0.7211102,0.6500001,0.4596194,0.9192388,0.37527764,1.1258893,0.7211102,0.6500001,0.45961946,0.9192388,0.3752777,1.1258893,0.72111017,0.6500001,0.45961946,0.9192388,0.3752777,1.1258893,0.72111017,0.6500001,0.45961946,0.9192388,0.3752777,1.1258893,0.72111017,0.8000001,0.5656855,1.131371,0.4618802,1.3857099,0.8717798,0.8000001,0.5656855,1.131371,0.4618802,1.3857099,0.8717798,0.80000013,0.5656855,1.131371,0.4618802,1.3857098,0.87177986,0.80000013,0.5656855,1.131371,0.4618802,1.3857098,0.87177986,0.95000005,0.6717515,1.343503,0.5484828,1.6455305,0.97467947,
0.1,0.28284273,0.14142136,0.099999994,0.28284273,0.14142138,0.099999994,0.2828427,0.14142138,0.099999994,0.28284273,0.14142135,0.099999994,0.28284273,0.14142135,0.10000001,0.2828427,0.14142138,0.100000024,0.2828427,0.14142138,0.100000024,0.28284276,0.14142138,0.100000024,0.2828427,0.14142138,0.100000024,0.28284276,0.14142135,0.100000024,0.28284276,0.14142135,0.100000024,0.28284276,0.14142132,0.100000024,0.28284276,0.14142132,0.100000024,0.28284276,0.14142132,0.100000024,0.28284276,0.14142132,0.100000024,0.28284276,0.14142132,0.100000024,0.2828427,0.14142132,0.100000024,0.28284276,0.14142132,0.100000024,0.28284276,0.14142138,0.1,0.28284273,0.14142136,0.099999994,0.28284273,0.14142138,0.099999994,0.2828427,0.14142138,0.099999994,0.28284273,0.14142135,0.099999994,0.28284273,0.14142135,0.10000001,0.2828427,0.14142138,0.100000024,0.2828427,0.14142138,0.100000024,0.28284276,0.14142138,0.100000024,0.2828427,0.14142138,0.100000024,0.28284276,0.14142135,0.100000024,0.28284276,0.14142135,0.100000024,0.28284276,0.14142132,0.100000024,0.28284276,0.14142132,0.100000024,0.28284276,0.14142132,0.100000024,0.28284276,0.14142132,0.100000024,0.28284276,0.14142132,0.100000024,0.2828427,0.14142132,0.100000024,0.28284276,0.14142132,0.100000024,0.28284276,0.14142138,0.1,0.28284273,0.14142136,0.099999994,0.28284273,0.14142138,0.099999994,0.2828427,0.14142138,0.099999994,0.28284273,0.14142135,0.099999994,0.28284273,0.14142135,0.10000001,0.2828427,0.14142138,0.100000024,0.2828427,0.14142138,0.100000024,0.28284276,0.14142138,0.100000024,0.2828427,0.14142138,0.100000024,0.28284276,0.14142135,0.100000024,0.28284276,0.14142135,0.100000024,0.28284276,0.14142132,0.100000024,0.28284276,0.14142132,0.100000024,0.28284276,0.14142132,0.100000024,0.28284276,0.14142132,0.100000024,0.28284276,0.14142132,0.100000024,0.2828427,0.14142132,0.100000024,0.28284276,0.14142132,0.100000024,0.28284276,0.14142138,0.1,0.28284273,0.14142136,0.099999994,0.28284273,0.14142138,0.099999994,0.2828427,0.14142138,0.099999994,0.28284273,0.14142135,0.099999994,0.28284273,0.14142135,0.10000001,0.2828427,0.14142138,0.100000024,0.2828427,0.14142138,0.100000024,0.28284276,0.14142138,0.100000024,0.2828427,0.14142138,0.100000024,0.28284276,0.14142135,0.100000024,0.28284276,0.14142135,0.100000024,0.28284276,0.14142132,0.100000024,0.28284276,0.14142132,0.100000024,0.28284276,0.14142132,0.100000024,0.28284276,0.14142132,0.100000024,0.28284276,0.14142132,0.100000024,0.2828427,0.14142132,0.100000024,0.28284276,0.14142132,0.100000024,0.28284276,0.14142138,0.1,0.28284273,0.14142136,0.099999994,0.28284273,0.14142138,0.099999994,0.2828427,0.14142138,0.099999994,0.28284273,0.14142135,0.099999994,0.28284273,0.14142135,0.10000001,0.2828427,0.14142138,0.100000024,0.2828427,0.14142138,0.100000024,0.28284276,0.14142138,0.100000024,0.2828427,0.14142138,0.100000024,0.28284276,0.14142135,0.100000024,0.28284276,0.14142135,0.100000024,0.28284276,0.14142132,0.100000024,0.28284276,0.14142132,0.100000024,0.28284276,0.14142132,0.100000024,0.28284276,0.14142132,0.100000024,0.28284276,0.14142132,0.100000024,0.2828427,0.14142132,0.100000024,0.28284276,0.14142132,0.100000024,0.28284276,0.14142138,0.1,0.28284273,0.14142136,0.099999994,0.28284273,0.14142138,0.099999994,0.2828427,0.14142138,0.099999994,0.28284273,0.14142135,0.099999994,0.28284273,0.14142135,0.10000001,0.2828427,0.14142138,0.100000024,0.2828427,0.14142138,0.100000024,0.28284276,0.14142138,0.100000024,0.2828427,0.14142138,0.100000024,0.28284276,0.14142135,0.100000024,0.28284276,0.14142135,0.100000024,0.28284276,0.14142132,0.100000024,0.28284276,0.14142132,0.100000024,0.28284276,0.14142132,0.100000024,0.28284276,0.14142132,0.100000024,0.28284276,0.14142132,0.100000024,0.2828427,0.14142132,0.100000024,0.28284276,0.14142132,0.100000024,0.28284276,0.14142138,0.1,0.28284273,0.14142136,0.099999994,0.28284273,0.14142138,0.099999994,0.2828427,0.14142138,0.099999994,0.28284273,0.14142135,0.099999994,0.28284273,0.14142135,0.10000001,0.2828427,0.14142138,0.100000024,0.2828427,0.14142138,0.100000024,0.28284276,0.14142138,0.100000024,0.2828427,0.14142138,0.100000024,0.28284276,0.14142135,0.100000024,0.28284276,0.14142135,0.100000024,0.28284276,0.14142132,0.100000024,0.28284276,0.14142132,0.100000024,0.28284276,0.14142132,0.100000024,0.28284276,0.14142132,0.100000024,0.28284276,0.14142132,0.100000024,0.2828427,0.14142132,0.100000024,0.28284276,0.14142132,0.100000024,0.28284276,0.14142138,0.1,0.28284273,0.14142136,0.099999994,0.28284273,0.14142138,0.099999994,0.2828427,0.14142138,0.099999994,0.28284273,0.14142135,0.099999994,0.28284273,0.14142135,0.10000001,0.2828427,0.14142138,0.100000024,0.2828427,0.14142138,0.100000024,0.28284276,0.14142138,0.100000024,0.2828427,0.14142138,0.100000024,0.28284276,0.14142135,0.100000024,0.28284276,0.14142135,0.100000024,0.28284276,0.14142132,0.100000024,0.28284276,0.14142132,0.100000024,0.28284276,0.14142132,0.100000024,0.28284276,0.14142132,0.100000024,0.28284276,0.14142132,0.100000024,0.2828427,0.14142132,0.100000024,0.28284276,0.14142132,0.100000024,0.28284276,0.14142138,0.1,0.28284273,0.14142136,0.099999994,0.28284273,0.14142138,0.099999994,0.2828427,0.14142138,0.099999994,0.28284273,0.14142135,0.099999994,0.28284273,0.14142135,0.10000001,0.2828427,0.14142138,0.100000024,0.2828427,0.14142138,0.100000024,0.28284276,0.14142138,0.100000024,0.2828427,0.14142138,0.100000024,0.28284276,0.14142135,0.100000024,0.28284276,0.14142135,0.100000024,0.28284276,0.14142132,0.100000024,0.28284276,0.14142132,0.100000024,0.28284276,0.14142132,0.100000024,0.28284276,0.14142132,0.100000024,0.28284276,0.14142132,0.100000024,0.2828427,0.14142132,0.100000024,0.28284276,0.14142132,0.100000024,0.28284276,0.14142138,0.1,0.28284273,0.14142136,0.099999994,0.28284273,0.14142138,0.099999994,0.2828427,0.14142138,0.099999994,0.28284273,0.14142135,0.099999994,0.28284273,0.14142135,0.10000001,0.2828427,0.14142138,0.100000024,0.2828427,0.14142138,0.100000024,0.28284276,0.14142138,0.100000024,0.2828427,0.14142138,0.100000024,0.28284276,0.14142135,0.100000024,0.28284276,0.14142135,0.100000024,0.28284276,0.14142132,0.100000024,0.28284276,0.14142132,0.100000024,0.28284276,0.14142132,0.100000024,0.28284276,0.14142132,0.100000024,0.28284276,0.14142132,0.100000024,0.2828427,0.14142132,0.100000024,0.28284276,0.14142132,0.100000024,0.28284276,0.14142138,0.1,0.28284273,0.14142136,0.099999994,0.28284273,0.14142138,0.099999994,0.2828427,0.14142138,0.099999994,0.28284273,0.14142135,0.099999994,0.28284273,0.14142135,0.10000001,0.2828427,0.14142138,0.100000024,0.2828427,0.14142138,0.100000024,0.28284276,0.14142138,0.100000024,0.2828427,0.14142138,0.100000024,0.28284276,0.14142135,0.100000024,0.28284276,0.14142135,0.100000024,0.28284276,0.14142132,0.100000024,0.28284276,0.14142132,0.100000024,0.28284276,0.14142132,0.100000024,0.28284276,0.14142132,0.100000024,0.28284276,0.14142132,0.100000024,0.2828427,0.14142132,0.100000024,0.28284276,0.14142132,0.100000024,0.28284276,0.14142138,0.1,0.28284273,0.14142136,0.099999994,0.28284273,0.14142138,0.099999994,0.2828427,0.14142138,0.099999994,0.28284273,0.14142135,0.099999994,0.28284273,0.14142135,0.10000001,0.2828427,0.14142138,0.100000024,0.2828427,0.14142138,0.100000024,0.28284276,0.14142138,0.100000024,0.2828427,0.14142138,0.100000024,0.28284276,0.14142135,0.100000024,0.28284276,0.14142135,0.100000024,0.28284276,0.14142132,0.100000024,0.28284276,0.14142132,0.100000024,0.28284276,0.14142132,0.100000024,0.28284276,0.14142132,0.100000024,0.28284276,0.14142132,0.100000024,0.2828427,0.14142132,0.100000024,0.28284276,0.14142132,0.100000024,0.28284276,0.14142138,0.1,0.28284273,0.14142136,0.099999994,0.28284273,0.14142138,0.099999994,0.2828427,0.14142138,0.099999994,0.28284273,0.14142135,0.099999994,0.28284273,0.14142135,0.10000001,0.2828427,0.14142138,0.100000024,0.2828427,0.14142138,0.100000024,0.28284276,0.14142138,0.100000024,0.2828427,0.14142138,0.100000024,0.28284276,0.14142135,0.100000024,0.28284276,0.14142135,0.100000024,0.28284276,0.14142132,0.100000024,0.28284276,0.14142132,0.100000024,0.28284276,0.14142132,0.100000024,0.28284276,0.14142132,0.100000024,0.28284276,0.14142132,0.100000024,0.2828427,0.14142132,0.100000024,0.28284276,0.14142132,0.100000024,0.28284276,0.14142138,0.1,0.28284273,0.14142136,0.099999994,0.28284273,0.14142138,0.099999994,0.2828427,0.14142138,0.099999994,0.28284273,0.14142135,0.099999994,0.28284273,0.14142135,0.10000001,0.2828427,0.14142138,0.100000024,0.2828427,0.14142138,0.100000024,0.28284276,0.14142138,0.100000024,0.2828427,0.14142138,0.100000024,0.28284276,0.14142135,0.100000024,0.28284276,0.14142135,0.100000024,0.28284276,0.14142132,0.100000024,0.28284276,0.14142132,0.100000024,0.28284276,0.14142132,0.100000024,0.28284276,0.14142132,0.100000024,0.28284276,0.14142132,0.100000024,0.2828427,0.14142132,0.100000024,0.28284276,0.14142132,0.100000024,0.28284276,0.14142138,0.1,0.28284273,0.14142136,0.099999994,0.28284273,0.14142138,0.099999994,0.2828427,0.14142138,0.099999994,0.28284273,0.14142135,0.099999994,0.28284273,0.14142135,0.10000001,0.2828427,0.14142138,0.100000024,0.2828427,0.14142138,0.100000024,0.28284276,0.14142138,0.100000024,0.2828427,0.14142138,0.100000024,0.28284276,0.14142135,0.100000024,0.28284276,0.14142135,0.100000024,0.28284276,0.14142132,0.100000024,0.28284276,0.14142132,0.100000024,0.28284276,0.14142132,0.100000024,0.28284276,0.14142132,0.100000024,0.28284276,0.14142132,0.100000024,0.2828427,0.14142132,0.100000024,0.28284276,0.14142132,0.100000024,0.28284276,0.14142138,0.1,0.28284273,0.14142136,0.099999994,0.28284273,0.14142138,0.099999994,0.2828427,0.14142138,0.099999994,0.28284273,0.14142135,0.099999994,0.28284273,0.14142135,0.10000001,0.2828427,0.14142138,0.100000024,0.2828427,0.14142138,0.100000024,0.28284276,0.14142138,0.100000024,0.2828427,0.14142138,0.100000024,0.28284276,0.14142135,0.100000024,0.28284276,0.14142135,0.100000024,0.28284276,0.14142132,0.100000024,0.28284276,0.14142132,0.100000024,0.28284276,0.14142132,0.100000024,0.28284276,0.14142132,0.100000024,0.28284276,0.14142132,0.100000024,0.2828427,0.14142132,0.100000024,0.28284276,0.14142132,0.100000024,0.28284276,0.14142138,0.1,0.28284273,0.14142136,0.099999994,0.28284273,0.14142138,0.099999994,0.2828427,0.14142138,0.099999994,0.28284273,0.14142135,0.099999994,0.28284273,0.14142135,0.10000001,0.2828427,0.14142138,0.100000024,0.2828427,0.14142138,0.100000024,0.28284276,0.14142138,0.100000024,0.2828427,0.14142138,0.100000024,0.28284276,0.14142135,0.100000024,0.28284276,0.14142135,0.100000024,0.28284276,0.14142132,0.100000024,0.28284276,0.14142132,0.100000024,0.28284276,0.14142132,0.100000024,0.28284276,0.14142132,0.100000024,0.28284276,0.14142132,0.100000024,0.2828427,0.14142132,0.100000024,0.28284276,0.14142132,0.100000024,0.28284276,0.14142138,0.1,0.28284273,0.14142136,0.099999994,0.28284273,0.14142138,0.099999994,0.2828427,0.14142138,0.099999994,0.28284273,0.14142135,0.099999994,0.28284273,0.14142135,0.10000001,0.2828427,0.14142138,0.100000024,0.2828427,0.14142138,0.100000024,0.28284276,0.14142138,0.100000024,0.2828427,0.14142138,0.100000024,0.28284276,0.14142135,0.100000024,0.28284276,0.14142135,0.100000024,0.28284276,0.14142132,0.100000024,0.28284276,0.14142132,0.100000024,0.28284276,0.14142132,0.100000024,0.28284276,0.14142132,0.100000024,0.28284276,0.14142132,0.100000024,0.2828427,0.14142132,0.100000024,0.28284276,0.14142132,0.100000024,0.28284276,0.14142138,0.1,0.28284273,0.14142136,0.099999994,0.28284273,0.14142138,0.099999994,0.2828427,0.14142138,0.099999994,0.28284273,0.14142135,0.099999994,0.28284273,0.14142135,0.10000001,0.2828427,0.14142138,0.100000024,0.2828427,0.14142138,0.100000024,0.28284276,0.14142138,0.100000024,0.2828427,0.14142138,0.100000024,0.28284276,0.14142135,0.100000024,0.28284276,0.14142135,0.100000024,0.28284276,0.14142132,0.100000024,0.28284276,0.14142132,0.100000024,0.28284276,0.14142132,0.100000024,0.28284276,0.14142132,0.100000024,0.28284276,0.14142132,0.100000024,0.2828427,0.14142132,0.100000024,0.28284276,0.14142132,0.100000024,0.28284276,0.14142138,0.34999996,0.4949747,0.24748735,0.60621774,0.20206249,0.41833,0.34999996,0.49497467,0.24748737,0.60621774,0.20206249,0.41833,0.34999996,0.49497473,0.24748737,0.60621774,0.20206249,0.41833,0.34999993,0.49497473,0.24748737,0.60621774,0.20206249,0.41832998,0.34999996,0.49497467,0.24748737,0.60621774,0.20206246,0.41833,0.35,0.49497473,0.24748734,0.60621774,0.20206249,0.41833004,0.35,0.49497473,0.2474873,0.60621774,0.20206249,0.41833004,0.3499999,0.49497473,0.2474873,0.6062178,0.20206249,0.41832995,0.3499999,0.49497467,0.2474873,0.6062177,0.20206249,0.41832995,0.3499999,0.49497467,0.2474873,0.6062178,0.20206255,0.41832995,0.34999996,0.4949747,0.24748735,0.60621774,0.20206249,0.41833,0.34999996,0.49497467,0.24748737,0.60621774,0.20206249,0.41833,0.34999996,0.49497473,0.24748737,0.60621774,0.20206249,0.41833,0.34999993,0.49497473,0.24748737,0.60621774,0.20206249,0.41832998,0.34999996,0.49497467,0.24748737,0.60621774,0.20206246,0.41833,0.35,0.49497473,0.24748734,0.60621774,0.20206249,0.41833004,0.35,0.49497473,0.2474873,0.60621774,0.20206249,0.41833004,0.3499999,0.49497473,0.2474873,0.6062178,0.20206249,0.41832995,0.3499999,0.49497467,0.2474873,0.6062177,0.20206249,0.41832995,0.3499999,0.49497467,0.2474873,0.6062178,0.20206255,0.41832995,0.34999996,0.4949747,0.24748735,0.60621774,0.20206249,0.41833,0.34999996,0.49497467,0.24748737,0.60621774,0.20206249,0.41833,0.34999996,0.49497473,0.24748737,0.60621774,0.20206249,0.41833,0.34999993,0.49497473,0.24748737,0.60621774,0.20206249,0.41832998,0.34999996,0.49497467,0.24748737,0.60621774,0.20206246,0.41833,0.35,0.49497473,0.24748734,0.60621774,0.20206249,0.41833004,0.35,0.49497473,0.2474873,0.60621774,0.20206249,0.41833004,0.3499999,0.49497473,0.2474873,0.6062178,0.20206249,0.41832995,0.3499999,0.49497467,0.2474873,0.6062177,0.20206249,0.41832995,0.3499999,0.49497467,0.2474873,0.6062178,0.20206255,0.41832995,0.34999996,0.4949747,0.24748735,0.60621774,0.20206249,0.41833,0.34999996,0.49497467,0.24748737,0.60621774,0.20206249,0.41833,0.34999996,0.49497473,0.24748737,0.60621774,0.20206249,0.41833,0.34999993,0.49497473,0.24748737,0.60621774,0.20206249,0.41832998,0.34999996,0.49497467,0.24748737,0.60621774,0.20206246,0.41833,0.35,0.49497473,0.24748734,0.60621774,0.20206249,0.41833004,0.35,0.49497473,0.2474873,0.60621774,0.20206249,0.41833004,0.3499999,0.49497473,0.2474873,0.6062178,0.20206249,0.41832995,0.3499999,0.49497467,0.2474873,0.6062177,0.20206249,0.41832995,0.3499999,0.49497467,0.2474873,0.6062178,0.20206255,0.41832995,0.34999996,0.4949747,0.24748735,0.60621774,0.20206249,0.41833,0.34999996,0.49497467,0.24748737,0.60621774,0.20206249,0.41833,0.34999996,0.49497473,0.24748737,0.60621774,0.20206249,0.41833,0.34999993,0.49497473,0.24748737,0.60621774,0.20206249,0.41832998,0.34999996,0.49497467,0.24748737,0.60621774,0.20206246,0.41833,0.35,0.49497473,0.24748734,0.60621774,0.20206249,0.41833004,0.35,0.49497473,0.2474873,0.60621774,0.20206249,0.41833004,0.3499999,0.49497473,0.2474873,0.6062178,0.20206249,0.41832995,0.3499999,0.49497467,0.2474873,0.6062177,0.20206249,0.41832995,0.3499999,0.49497467,0.2474873,0.6062178,0.20206255,0.41832995,0.34999996,0.4949747,0.24748735,0.60621774,0.20206249,0.41833,0.34999996,0.49497467,0.24748737,0.60621774,0.20206249,0.41833,0.34999996,0.49497473,0.24748737,0.60621774,0.20206249,0.41833,0.34999993,0.49497473,0.24748737,0.60621774,0.20206249,0.41832998,0.34999996,0.49497467,0.24748737,0.60621774,0.20206246,0.41833,0.35,0.49497473,0.24748734,0.60621774,0.20206249,0.41833004,0.35,0.49497473,0.2474873,0.60621774,0.20206249,0.41833004,0.3499999,0.49497473,0.2474873,0.6062178,0.20206249,0.41832995,0.3499999,0.49497467,0.2474873,0.6062177,0.20206249,0.41832995,0.3499999,0.49497467,0.2474873,0.6062178,0.20206255,0.41832995,0.34999996,0.4949747,0.24748735,0.60621774,0.20206249,0.41833,0.34999996,0.49497467,0.24748737,0.60621774,0.20206249,0.41833,0.34999996,0.49497473,0.24748737,0.60621774,0.20206249,0.41833,0.34999993,0.49497473,0.24748737,0.60621774,0.20206249,0.41832998,0.34999996,0.49497467,0.24748737,0.60621774,0.20206246,0.41833,0.35,0.49497473,0.24748734,0.60621774,0.20206249,0.41833004,0.35,0.49497473,0.2474873,0.60621774,0.20206249,0.41833004,0.3499999,0.49497473,0.2474873,0.6062178,0.20206249,0.41832995,0.3499999,0.49497467,0.2474873,0.6062177,0.20206249,0.41832995,0.3499999,0.49497467,0.2474873,0.6062178,0.20206255,0.41832995,0.34999996,0.4949747,0.24748735,0.60621774,0.20206249,0.41833,0.34999996,0.49497467,0.24748737,0.60621774,0.20206249,0.41833,0.34999996,0.49497473,0.24748737,0.60621774,0.20206249,0.41833,0.34999993,0.49497473,0.24748737,0.60621774,0.20206249,0.41832998,0.34999996,0.49497467,0.24748737,0.60621774,0.20206246,0.41833,0.35,0.49497473,0.24748734,0.60621774,0.20206249,0.41833004,0.35,0.49497473,0.2474873,0.60621774,0.20206249,0.41833004,0.3499999,0.49497473,0.2474873,0.6062178,0.20206249,0.41832995,0.3499999,0.49497467,0.2474873,0.6062177,0.20206249,0.41832995,0.3499999,0.49497467,0.2474873,0.6062178,0.20206255,0.41832995,0.34999996,0.4949747,0.24748735,0.60621774,0.20206249,0.41833,0.34999996,0.49497467,0.24748737,0.60621774,0.20206249,0.41833,0.34999996,0.49497473,0.24748737,0.60621774,0.20206249,0.41833,0.34999993,0.49497473,0.24748737,0.60621774,0.20206249,0.41832998,0.34999996,0.49497467,0.24748737,0.60621774,0.20206246,0.41833,0.35,0.49497473,0.24748734,0.60621774,0.20206249,0.41833004,0.35,0.49497473,0.2474873,0.60621774,0.20206249,0.41833004,0.3499999,0.49497473,0.2474873,0.6062178,0.20206249,0.41832995,0.3499999,0.49497467,0.2474873,0.6062177,0.20206249,0.41832995,0.3499999,0.49497467,0.2474873,0.6062178,0.20206255,0.41832995,0.34999996,0.4949747,0.24748735,0.60621774,0.20206249,0.41833,0.34999996,0.49497467,0.24748737,0.60621774,0.20206249,0.41833,0.34999996,0.49497473,0.24748737,0.60621774,0.20206249,0.41833,0.34999993,0.49497473,0.24748737,0.60621774,0.20206249,0.41832998,0.34999996,0.49497467,0.24748737,0.60621774,0.20206246,0.41833,0.35,0.49497473,0.24748734,0.60621774,0.20206249,0.41833004,0.35,0.49497473,0.2474873,0.60621774,0.20206249,0.41833004,0.3499999,0.49497473,0.2474873,0.6062178,0.20206249,0.41832995,0.3499999,0.49497467,0.2474873,0.6062177,0.20206249,0.41832995,0.3499999,0.49497467,0.2474873,0.6062178,0.20206255,0.41832995,0.49999997,0.7071067,0.35355335,0.8660254,0.2886607,0.57008773,0.5,0.7071067,0.35355335,0.8660253,0.2886607,0.5700878,0.5,0.7071067,0.35355332,0.86602545,0.28866073,0.5700877,0.5,0.70710665,0.3535533,0.86602545,0.28866076,0.5700877,0.49999994,0.7071067,0.3535534,0.8660253,0.28866065,0.5700878,0.49999997,0.7071067,0.35355335,0.8660254,0.2886607,0.57008773,0.5,0.7071067,0.35355335,0.8660253,0.2886607,0.5700878,0.5,0.7071067,0.35355332,0.86602545,0.28866073,0.5700877,0.5,0.70710665,0.3535533,0.86602545,0.28866076,0.5700877,0.49999994,0.7071067,0.3535534,0.8660253,0.28866065,0.5700878,0.49999997,0.7071067,0.35355335,0.8660254,0.2886607,0.57008773,0.5,0.7071067,0.35355335,0.8660253,0.2886607,0.5700878,0.5,0.7071067,0.35355332,0.86602545,0.28866073,0.5700877,0.5,0.70710665,0.3535533,0.86602545,0.28866076,0.5700877,0.49999994,0.7071067,0.3535534,0.8660253,0.28866065,0.5700878,0.49999997,0.7071067,0.35355335,0.8660254,0.2886607,0.57008773,0.5,0.7071067,0.35355335,0.8660253,0.2886607,0.5700878,0.5,0.7071067,0.35355332,0.86602545,0.28866073,0.5700877,0.5,0.70710665,0.3535533,0.86602545,0.28866076,0.5700877,0.49999994,0.7071067,0.3535534,0.8660253,0.28866065,0.5700878,0.49999997,0.7071067,0.35355335,0.8660254,0.2886607,0.57008773,0.5,0.7071067,0.35355335,0.8660253,0.2886607,0.5700878,0.5,0.7071067,0.35355332,0.86602545,0.28866073,0.5700877,0.5,0.70710665,0.3535533,0.86602545,0.28866076,0.5700877,0.49999994,0.7071067,0.3535534,0.8660253,0.28866065,0.5700878,0.6499999,0.9192387,0.45961934,1.1258329,0.3752589,0.7211102,0.64999986,0.9192387,0.4596193,1.125833,0.37525892,0.7211102,0.64999986,0.91923875,0.45961928,1.1258328,0.37525892,0.72111017,0.6499999,0.9192387,0.45961934,1.1258329,0.3752589,0.7211102,0.64999986,0.9192387,0.4596193,1.125833,0.37525892,0.7211102,0.64999986,0.91923875,0.45961928,1.1258328,0.37525892,0.72111017,0.6499999,0.9192387,0.45961934,1.1258329,0.3752589,0.7211102,0.64999986,0.9192387,0.4596193,1.125833,0.37525892,0.7211102,0.64999986,0.91923875,0.45961928,1.1258328,0.37525892,0.72111017,0.79999995,1.1313708,0.5656854,1.3856406,0.46185714,0.8717798,0.79999995,1.1313708,0.56568533,1.3856406,0.46185708,0.87177986,0.79999995,1.1313708,0.5656854,1.3856406,0.46185714,0.8717798,0.79999995,1.1313708,0.56568533,1.3856406,0.46185708,0.87177986,0.9499999,1.3435028,0.6717514,1.6454482,0.54845536,0.97467947
]

In [6]:
CLASS_NUM = 21
RESULT_NUM = 1917
X_SCALE = 10
Y_SCALE = 10
W_SCALE = 5
H_SCALE = 5
IMG_DIMENSION = 300
import math
def getbox(input, region_id):
    x_center, y_center = 0.0, 0.0
    w, h = 0.0, 0.0
    bbox = [-1,-1,-1,-1]
    prior_x = _box_prior[1 * RESULT_NUM + region_id]
    prior_y = _box_prior[0 * RESULT_NUM + region_id]
    prior_w = _box_prior[3 * RESULT_NUM + region_id]
    prior_h = _box_prior[2 * RESULT_NUM + region_id]

    y_center = input[0] / Y_SCALE * prior_h + prior_y
    x_center = input[1] / X_SCALE * prior_w + prior_x

    h = math.exp(input[2] / H_SCALE) * prior_h
    w = math.exp(input[3] / W_SCALE) * prior_w

    bbox[0] = IMG_DIMENSION * (x_center - w / 2)
    bbox[1] = IMG_DIMENSION * (y_center - h / 2)
    bbox[2] = IMG_DIMENSION * w
    bbox[3] = IMG_DIMENSION * h
    return bbox

def getconfidence(input):
    max_val = -1.0
    adjusted = 0.0
    class_index = 0
    confidence = 0
    # Skip the first catch-all class
    input = input[1:]

    for i in range(0, CLASS_NUM-1):
        data = input[i]
        adjusted = 1 / (1 + math.exp(-1*data))

        if adjusted > max_val:
            max_val = adjusted
            class_index = i

    confidence = max_val
    ans = [class_index,confidence]
    
    return ans

def clip_window(data):
    if data is None:
        return -1
  
    wx1 = 0
    wx2 = 300
    wy1 = 0
    wy2 = 300

    bx1 = data[0]
    by1 = data[1]
    bx2 = data[0] + data[2]
    by2 = data[1] + data[3]

    if bx1 < wx1:
        bx1 = wx1
    if bx2 < wx1:
        bx2 = wx1
    if bx1 > wx2:
        bx1 = wx2
    if bx2 > wx2:
        bx2 = wx2
    if by1 < wy1:
        by1 = wy1
    if by2 < wy1:
        by2 = wy1
    if by1 > wy2:
        by1 = wy2
    if by2 > wy2:
        by2 = wy2
    newdata = [-1,-1,-1,-1]
    newdata[0] = bx1
    newdata[1] = by1
    newdata[2] = bx2 - bx1
    newdata[3] = by2 - by1
    invalid = False
    if newdata[2] * newdata[3] < 1e-5:
        invalid = True

    return newdata, invalid

def classbox_compare(a, b):
    if a[3]:
        if b[3]:
            return 0
        else:
            return 1
    if b[3]:
        return -1
    return -1 if a[2] < b[2] else 1

def resultbox_compare(a,b):
    x = a
    y = b
    
    if x == -1:
        if y == -1:
            return 0
        else:
            return 1
    
    if y == -1:
        return -1
    
    return 1 if x[2] < y[2] else -1

def MIN(x, y):
    return y if x > y else x

def MAX(x, y):
    return y if x < y else x

def IS_VALID_RECT(bbox):
    return bbox[2] > 0 and bbox[3] > 0

def RECT_AREA(bbox):
    return bbox[2] * bbox[3]

def get_iou(box_a, box_b):
    overlap = [-1,-1,-1,-1]

    overlap[0] = MAX(box_a[0], box_b[0])
    overlap[1] = MAX(box_a[1], box_b[1])

    overlap[2] = MIN(box_a[0] + box_a[2], box_b[0] + box_b[2]) - overlap[0]
    overlap[3] = MIN(box_a[1] + box_a[3], box_b[1] + box_b[3]) - overlap[1]

    if not IS_VALID_RECT(overlap):
        return 0.0

    return RECT_AREA(overlap) / (RECT_AREA(box_a) + RECT_AREA(box_b) - RECT_AREA(overlap))

MIN_CONFIDENCE = 0.5
IOU_THRESHOLD = 0.6

def NMS_byclass(classboxlist, boxlist, classid, current_id, boxes):
    if classboxlist[current_id][1] != classid:
        return -1
    
    curid = current_id
    next_item = -1
    indexes = [] # invalid indexes for classboxlist
    
    for i in range(curid + 1, RESULT_NUM):
        if classboxlist[i][3]:
            continue
        
        if classboxlist[i][2] < MIN_CONFIDENCE:
            indexes.append(i)
            #classboxlist[i][3] = True
            continue
        
        boxid = classboxlist[i][0]
        curboxid = classboxlist[curid][0]
        #if boxlist[boxid][3]:
        if boxlist[boxid] == True:
            indexes.append(i)
            #classboxlist[i][3] = True
            continue
        
        if classboxlist[i][1] == classid and get_iou(boxes[boxid], boxes[curboxid]) > IOU_THRESHOLD:
            indexes.append(i)
            #classboxlist[i][3] = True
            continue
        
        if next_item < 0:
            next_item = i
    
    return next_item, indexes

In [8]:
labels = [
    "???",
    "aeroplane",
    "bicycle",
    "bird",
    "boat",
    "bottle",
    "bus",
    "car",
    "cat",
    "chair",
    "cow",
    "diningtable",
    "dog",
    "horse",
    "motorbike",
    "person",
    "pottedplant",
    "sheep",
    "sofa",
    "train",
    "tvmonitor"
]
interpreter = tf.lite.Interpreter(model_path="./tflite_models/mobilenet_ssd_pascal_quant.tflite")
interpreter.allocate_tensors()
output = interpreter.get_output_details()
input = interpreter.get_input_details()
output_index1 = output[0]['index']
output_index2 = output[1]['index']
input_index = input[0]['index']


from PIL import Image
vid_path = "./posevid.mp4"
cap = cv2.VideoCapture(vid_path)
ret, frame = cap.read()
alldata = []
for h in range(0,250): 
    width, height = get_video_dimensions(vid_path) 
    cropped_frame = crop_frame(frame, height, height)
    frame = cropped_frame
    resized_frame = cv2.resize(frame, (300, 300))
    #cv2.imwrite("./resized_image.jpg", resized_frame)
    color_frame = cv2.cvtColor(resized_frame, cv2.COLOR_BGR2RGB)
    frame = np.expand_dims(color_frame, axis=0)
    interpreter.set_tensor(input_index, frame)
    interpreter.invoke()
    output_data1 = interpreter.get_tensor(output_index1)
    output_data2 = interpreter.get_tensor(output_index2)
    # print(output_data1[0][0])
    # print(output_data2.shape)
    MIN_CONFIDENCE = 0.5
    TOP_N_INCLASS = 2
    ALL_RESULT_SIZE = TOP_N_INCLASS*CLASS_NUM
    cols0 = 4
    rows0 = 1917
    boxdata = [[-1 for _ in range(cols0)] for _ in range(rows0)]
    #print(np.array(boxdata).shape)
    classdata = [] # [class_id, confidence]
    cols = 1917
    rows = 21
    classboxlist = [[-1 for _ in range(cols)] for _ in range(rows)] # size 21 [1st class: [(box_id, class_id, confidence, invalid)], [1st info],... [1917th info]
                                                                    #           2nd class:                                         ]
    #classresultlist = [[-1 for _ in range(4)] for _ in range(ALL_RESULT_SIZE)]
    classresultlist = [-1]*ALL_RESULT_SIZE

    import time
    DEQUANT_BBOX_MEAN = 173
    DEQUANT_BBOX_STEP = 0.064610
    DEQUANT_CLASS_MEAN = 159
    DEQUANT_CLASS_STEP = 0.141151
    dequant_box = output_data2.astype(float)
    dequant_class = output_data1.astype(float)
    for a in range(0,1917):
        for b in range(0,4):
            newval = ((output_data2[0][a][b]).astype(float)- DEQUANT_BBOX_MEAN)*DEQUANT_BBOX_STEP
            dequant_box[0][a][b] = newval
        for c in range(0,21):
            newval = ((output_data1[0][a][c]).astype(float) - DEQUANT_CLASS_MEAN)*DEQUANT_CLASS_STEP
            dequant_class[0][a][c] = newval

    # dequant_box = [(x - DEQUANT_BBOX_MEAN)*DEQUANT_BBOX_STEP for x in output_data2]
    # dequant_class = [(x - DEQUANT_CLASS_MEAN)*DEQUANT_CLASS_STEP for x in output_data1]
    invalidindexes= [False] * 1917 # 0 = not invalid, -1 = invalid for &data
    for i in range(0,1917):
        boxdata[i] = (getbox(dequant_box[0][i], i))
        classdata.append(getconfidence(dequant_class[0][i]))
        for j in range(1, CLASS_NUM):
            info = (i,j,1 / (1 + math.exp(-1*dequant_class[0][i][j])),False)
            classboxlist[j][i] = info
        clip_window_output = clip_window(boxdata[i])
        boxdata[i] = clip_window_output[0]
        invalidindexes[i] = clip_window_output[1]
    import functools
    #NMS
    for k in range(1, CLASS_NUM):#1->21
        current_item = 0
        # Sort the result (descending)
        classboxlist[k] = sorted(classboxlist[k], key=functools.cmp_to_key(classbox_compare), reverse=True)
        
        if classboxlist[k][0][3] == True:
            continue
        
        if classboxlist[k][0][2] < MIN_CONFIDENCE:
            oldtup = classboxlist[k][0]
            newtup = (oldtup[0],oldtup[1],oldtup[2],True)
            classboxlist[k][0] = newtup
            continue

        for l in range(0,TOP_N_INCLASS):#0->2
            if current_item >= RESULT_NUM:
                break
            # Perform non-maximum suppression and get the next item to be output
            NMS_output = NMS_byclass(classboxlist[k],invalidindexes,k,current_item,boxdata)
            next_item = NMS_output[0]
            invalids = NMS_output[1]
            for z in range(0,len(invalids)):
                oldtup = classboxlist[k][invalids[z]]
                newtup = (oldtup[0],oldtup[1],oldtup[2],True)
                classboxlist[k][invalids[z]] = newtup
            classresultlist[k * TOP_N_INCLASS + l] = classboxlist[k][current_item]
            # No more valid output
            if next_item < 0:
                break

            current_item = next_item

    rescnt = 0
    for i in range(ALL_RESULT_SIZE):
        if classresultlist[i] != -1:
            classresultlist[rescnt] = classresultlist[i]
            rescnt += 1

    #print("Detections: "+str(rescnt))
    classresultlist = classresultlist[:rescnt]
    #print(classresultlist)
    TOP_N_RESULT = 2
    # Sort the classresultlist
    classresultlist = sorted(classresultlist, key=functools.cmp_to_key(resultbox_compare), reverse=True)
    outputsize = 0
    if rescnt > TOP_N_RESULT:
        outputsize = TOP_N_RESULT
    else:
        outputsize = rescnt

    #print("Top "+str(outputsize)+" Results:")
    #[(box_id, class_id, confidence, invalid)]
    for i in range(outputsize):
        pclassbox = classresultlist[i]
        if pclassbox == -1:
            continue
        current_item = pclassbox[0]
        if pclassbox[3] == False and invalidindexes[current_item] == False:
            if current_item < 0 or current_item > RESULT_NUM:
                print("Something wrong")
            # output msg
            framedata = []
            framedata.append((int)(boxdata[current_item][0]))
            framedata.append((int)(boxdata[current_item][1]))
            framedata.append((int)(boxdata[current_item][0]+boxdata[current_item][2]))
            framedata.append((int)(boxdata[current_item][1]+boxdata[current_item][3]))
            alldata.append(framedata)
            #return_msg = str(labels[pclassbox[1]])+" (ID: "+str(pclassbox[1])+"): "+"\nConfidence: "+str(pclassbox[2])+"\nBBOx: "+str(boxdata[current_item][0])+", "+str(boxdata[current_item][1])+", "+str(boxdata[current_item][2])+", "+str(boxdata[current_item][3])
            #print(return_msg)
    

    ret, frame = cap.read()

    # for i in range(0,len(items)):
    #     cv2.rectangle(image, ((int)(boxdata[items[i]][0]),(int)(boxdata[items[i]][1])),((int)(boxdata[items[i]][0]+boxdata[items[i]][2]),(int)(boxdata[items[i]][1]+boxdata[items[i]][3])), color, thickness)
    # # Display the image with the bounding box
    # cv2.imshow("Image with Bounding Box", image)
    # cv2.waitKey(0)
    # cv2.destroyAllWindows()

In [9]:
print(len(alldata))


441


In [48]:
#newarr = np.array(AllPoints)
print(len(AllPoints[0]))
#print(newarr.shape)
# AllPoints = [[1, 2, 3], [4, 5, 6], [7, 8, 9]]

# Specify the file path and name
file_path = "./test.txt"

# Open the file in write mode
with open(file_path, 'w') as file:
    # Iterate over each sublist in AllPoints
    for sublist in AllPoints:
        # Convert the sublist to a string
        sublist_str = ' '.join(map(str, sublist))
        # Write the sublist string to the file
        file.write(sublist_str + '\n')

# Specify the file path and name
file_path = "./test.txt"

# Initialize the larger list
larger_list = []

# Read the contents of the file
with open(file_path, 'r') as file:
    # Iterate over each line in the file
    for line in file:
        # Strip any leading/trailing whitespaces and newline characters
        line = line.strip()
        line = line.replace(", ",",")
        # Split the line by spaces to separate the elements
        elements_str = line.split()
        
        # Ensure the line has exactly 18 elements
        if len(elements_str) == 18:
            # Convert elements to tuples
            elements = [tuple(map(int, elem.strip('()').split(','))) for elem in elements_str]
            
            # Append the elements list to the larger list
            larger_list.append(elements)
print(larger_list[0])
new_large = []

for row in larger_list:
    new_row = []
    for pt in row:
        if pt[0] != -1:
            newx = (int)(pt[0]/46*900)
            newy = (int)(pt[1]/46*900)
            pt = (newx,newy) 
            new_row.append(pt)
        else:
            new_row.append((-1,-1))
    new_large.append(new_row)
print(new_large[0])
print(new_large[1])



NameError: name 'AllPoints' is not defined

## Prepare data for model 2

In [52]:
moving_probs_trimmed = moving_prob[:-(len(moving_prob)%16)] if len(moving_prob) % 16 != 0 else moving_prob
model2_in = np.array(moving_probs_trimmed).reshape((len(moving_prob)//16, 16))
# print(model2_in)

## Pass into Model 2 and get final prediction

In [53]:
model2 = tf.keras.models.load_model("model2")

In [54]:
vid_preds = model2.predict(model2_in)

In [55]:
bools = vid_preds.reshape((1, len(vid_preds))) > 0.5
print(bools)

[[ True  True False False False False False False False False False]]
